# import Library

In [1]:
import ccxt
# CCXT 버전을 최신으로 유지해야 hyperliquid 기능이 정상 작동할 수 있습니다.
print('CCXT Version:', ccxt.__version__)
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta
from decimal import Decimal
from tqdm.notebook import tqdm  
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from itertools import combinations, permutations
import time
import os
from dotenv import load_dotenv
from typing import Dict, List
from IPython.display import display

CCXT Version: 4.4.59


In [2]:
def allprint(df):
    """
    pandas DataFrame의 모든 행과 모든 열을 제한 없이 출력하는 함수.
    사용 후 원래 display 설정을 복원함.
    """
    # 현재 설정값 백업
    old_max_rows = pd.get_option('display.max_rows')
    old_max_cols = pd.get_option('display.max_columns')
    
    try:
        # 임시로 최대 행/열 출력 제한 해제
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        
        display(df)
    finally:
        # 함수 종료 후 원래 설정 복원
        pd.set_option('display.max_rows', old_max_rows)
        pd.set_option('display.max_columns', old_max_cols)



def precision_to_decimals(x):
    # float → 문자열 변환 후 Decimal로 변환하여 오차 방지
    d = Decimal(str(x)).normalize()
    # exponent가 -n이면 소수점 n자리 => decimals = -exponent
    return -d.as_tuple().exponent

def get_next_N_hours_intervals_from_now(utc_now: pd.Timestamp, N: int):
    """
    현재 utc_now 이후의 다음 정각부터 N시간 동안의 정각 시간대를 vectorized하게 반환합니다.
    
    :param utc_now: 현재 UTC 시간 (pd.Timestamp)
    :param N: 생성할 시간 슬롯의 개수 (예: 8)
    :return: pd.DatetimeIndex 객체 (각 정각 시간)
    """
    # utc_now 이후의 다음 정각 계산 (분, 초, 마이크로초 0)
    next_hour = (utc_now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
    # pd.date_range를 사용하여 정각 시간대를 생성 (freq='H'는 한 시간 간격)
    return pd.date_range(start=next_hour, periods=N, freq='h')

def reorder_columns(df: pd.DataFrame, priority_cols: list, level: int = None) -> pd.DataFrame:
    """
    DataFrame의 컬럼 순서를 지정한 priority_cols 리스트의 컬럼을 앞으로 배치하고, 
    나머지 컬럼은 기존 순서를 유지한 채 뒤에 배치하는 함수. 
    멀티인덱스에서 특정 레벨을 지정해 컬럼을 선택 가능.
    
    :param df: 정렬할 DataFrame
    :param priority_cols: 앞쪽에 배치할 컬럼 리스트 (단일 문자열, 튜플, 또는 레벨 값)
    :param level: 멀티인덱스에서 사용할 레벨 번호 (기본값 None: 전체 컬럼명 사용)
    :return: 컬럼이 재배치된 DataFrame
    """
    if isinstance(df.columns, pd.MultiIndex):
        all_cols = df.columns.tolist()  # 튜플 리스트: [(level1, level2), ...]
        if level is not None:
            level_values = df.columns.get_level_values(level).tolist()
            # priority_cols 순서대로 인덱스 찾기
            priority_indices = []
            for col in priority_cols:
                indices = [i for i, val in enumerate(level_values) if val == col]
                priority_indices.extend(indices)
            priority_full_cols = [all_cols[i] for i in priority_indices if all_cols[i] in all_cols]
        else:
            priority_full_cols = [col for col in priority_cols if col in all_cols]
    else:
        all_cols = df.columns.tolist()
        priority_full_cols = [col for col in priority_cols if col in all_cols]
    
    remaining_cols = [col for col in all_cols if col in df.columns and col not in priority_full_cols]
    new_order = priority_full_cols + remaining_cols
    return df[new_order]

def process_ticker_index(ticker_index):
    # (위에서 제공한 process_ticker_index 함수 그대로 사용)
    if isinstance(ticker_index, pd.Index):
        ticker = pd.Series(ticker_index.values, index=ticker_index)
    else:
        ticker = ticker_index.copy()

    index_split = ticker.str.replace(r'[:, /]', '|', regex=True).str.split('|', expand=True)
    base_coin = index_split[0]
    base_coin_array = base_coin.values

    scaling_factor = pd.Series(np.ones(base_coin_array.shape[0], dtype=int), index=ticker.index)
    scaling_factor[base_coin.str.startswith('k')] = 1000
    scaling_factor[base_coin.str.startswith('1M')] = 1000000
    numeric_match = base_coin.str.extract(r'^(?P<number>\d{2,})')['number']
    scaling_factor[numeric_match.notna()] = numeric_match.dropna().astype(int)

    index_k_or_number = base_coin.str.match(r'^(?:k|1M|\d{2,})')
    base_coin = pd.Series(
        np.where(index_k_or_number, 
                 base_coin.str.replace(r'^(?:k|1M|\d{2,})', '', regex=True), 
                 base_coin),
        index=ticker.index
    )
    quote_coin = index_split[1]

    return ticker.values, base_coin.values, quote_coin.values, scaling_factor.values


In [3]:
# 각 거래소별 API 키 설정 (본인의 키로 교체)
exchange_configs = {
    'binance': {
        'apiKey': os.getenv('BINANCE_API_KEY'),
        'secret': os.getenv('BINANCE_SECRET'),
        'options': {'defaultType': 'swap'}
    },
    'bybit': {
        'apiKey': os.getenv('BYBIT_API_KEY'),
        'secret': os.getenv('BYBIT_SECRET'),
        'options': {'defaultType': 'swap'}
    },
    'gateio': {
        'apiKey': os.getenv('GATEIO_API_KEY'),
        'secret': os.getenv('GATEIO_SECRET'),
        'options': {'defaultType': 'swap'}
    },
    
    'bitget': {
        'apiKey': os.getenv('BITGET_API_KEY'),
        'secret': os.getenv('BITGET_SECRET'),
        'password': os.getenv('BITGET_PASSWORD'),
        'options': {'defaultType': 'swap'}
    },
    #hyperliquid api key is vaild for 180 days
    'hyperliquid': {
        'walletAddress': os.getenv('HYPERLIQUID_WALLET_ADRESS'),
        'API_KEY': os.getenv('HYPERLIQUID_API_KEY'),
        'options': {'defaultType': 'swap'}
    }
}

# Fetcher Class

In [88]:
import ccxt
import pandas as pd
from datetime import datetime, timezone
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import requests
import numpy as np

class ExchangeDataFetcher:
    def __init__(self, exchanges_to_fetch=['bitget','hyperliquid', 'bybit', 'gateio', 'binance']):
        self.utc_now = pd.to_datetime(datetime.now(timezone.utc))
        self.exchanges = {}
        self.data_cache = {}
        self.exchanges_to_fetch = exchanges_to_fetch
        self.magin_dashboards = None

        with ThreadPoolExecutor(max_workers=len(exchanges_to_fetch)) as executor:
            futures = {executor.submit(self._preload_exchange_data, name): name for name in exchanges_to_fetch}
            for future in tqdm(as_completed(futures), total=len(futures), desc="Preloading Exchanges"):
                name = futures[future]
                try:
                    future.result()
                    print(f"✅ {name} 데이터 로드 완료")
                except Exception as e:
                    print(f"Error preloading {name}: {str(e)}")

        return None

    def initialize_exchange(self, name, config=None):
        """거래소 객체 초기화"""
        exchange_class = getattr(ccxt, name)
        config = config or {'enableRateLimit': True}
        self.exchanges[name] = exchange_class(config)
        return None

    def _preload_exchange_data(self, exchange_name):
        """모든 데이터를 병렬로 로드"""
        self.initialize_exchange(exchange_name, exchange_configs.get(exchange_name))
        exchange = self.exchanges[exchange_name]
        self.data_cache[exchange_name] = {}

        def load_markets_task():
            if exchange_name == 'gateio':
                #precision 정보를 추가하기 위해 load_markets를 수정
                ##info에서 quanto_multiplier를 가져오기
                gateio_load_markets=pd.DataFrame(exchange.load_markets())
                gateio_load_markets_info=gateio_load_markets.loc['info'].copy()
                gateio_quanto_multiplier=gateio_load_markets_info.apply(pd.Series)['quanto_multiplier'].dropna().astype('float64')

                ##precision 정보에 quanto_multiplier를 추가
                ###dict 변환
                gateio_load_markets_precision_df=gateio_load_markets.loc['precision'].apply(pd.Series)
                gateio_load_markets_precision_df_amount_series=gateio_load_markets_precision_df['amount'].copy()
                gateio_load_markets_precision_df_amount_series[gateio_quanto_multiplier.index]=gateio_quanto_multiplier
                gateio_load_markets_precision_df['amount']=gateio_load_markets_precision_df_amount_series
                gateio_precision_dict=gateio_load_markets_precision_df.T.to_dict()

                ##precision 정보에 quanto_multiplier를 추가
                gateio_load_markets.loc['precision']=gateio_precision_dict
                return pd.DataFrame(exchange.load_markets())
            else:
                return pd.DataFrame(exchange.load_markets())

        def fetch_funding_rates_task():
            return pd.DataFrame(exchange.fetch_funding_rates())
        
        def fetch_funding_rates_usdc_task():
            return pd.DataFrame(exchange.fetch_funding_rates(params={"productType": "USDC-FUTURES"}))

        def fetch_funding_intervals_task():
            return pd.DataFrame(exchange.fetch_funding_intervals()).loc['interval']

        def fetch_bids_asks_task():
            return pd.DataFrame(exchange.fetchBidsAsks(params={"type": "swap", "futures" : True}))

        def fetch_tickers_task():
            return pd.DataFrame(exchange.fetchTickers(params={"type": "swap"}))
        
        def fetch_balance_task():
            balance = exchange.fetch_balance()
            return balance
        
        def fetch_balance_usdc_task():
            balance = exchange.fetch_balance(params={"productType": "USDC-FUTURES"})
            return balance
        
        def fetch_positions_task():
            exchange_fetch_positions=exchange.fetch_positions()
            if len(exchange_fetch_positions)==0:
                return pd.DataFrame()
            
            elif exchange_name == 'binance':
                positions_df= pd.DataFrame(exchange.fetch_positions())
                positions_df['leverage'] = positions_df['notional']/positions_df['initialMargin']
            else:
                positions_df= pd.DataFrame(exchange.fetch_positions())

            symbol_series=positions_df['symbol']
            ticker_array, baseCoin_array, quoteCoin_array, scaling_factor=process_ticker_index(symbol_series)
            positions_df.index=baseCoin_array
            positions_df['baseCoin']=baseCoin_array
            positions_df['quoteCoin']=quoteCoin_array
            positions_df['scaling_factor']=scaling_factor
            return positions_df

        # 거래소별 tasks를 dict of dict로 정의
        exchange_tasks = {
            'binance': {
                'markets': load_markets_task,
                'funding_rates': fetch_funding_rates_task,
                'tickers': fetch_tickers_task,
                'funding_intervals': fetch_funding_intervals_task,
                'bids_asks': fetch_bids_asks_task,
                'balance': fetch_balance_task,
                'positions': fetch_positions_task,
            },
            'bitget': {
                'markets': load_markets_task,
                'funding_rates': fetch_funding_rates_task,
                'funding_rates_usdc': fetch_funding_rates_usdc_task,
                'balance': fetch_balance_task,
                'balance_usdc': fetch_balance_usdc_task,
                'positions': fetch_positions_task,
            },
            'hyperliquid': {
                'markets': load_markets_task,
                'funding_rates': fetch_funding_rates_task,
                'tickers': fetch_tickers_task,
                'balance': fetch_balance_task,
                'positions': fetch_positions_task,
            },
            'bybit': {
                'markets': load_markets_task,
                'funding_rates': fetch_funding_rates_task,
                'balance': fetch_balance_task,
                'positions': fetch_positions_task,
            },
            'gateio': {
                'markets': load_markets_task,
                'funding_rates': fetch_funding_rates_task,
                'tickers': fetch_tickers_task,
                'balance': fetch_balance_task,
                'positions': fetch_positions_task,
            }
        }

        tasks = exchange_tasks[exchange_name]
        with ThreadPoolExecutor(max_workers=len(tasks)) as executor:
            futures = {executor.submit(task): name for name, task in tasks.items()}
            for future in as_completed(futures):
                name = futures[future]
                max_attempts = 3  # 최대 시도 횟수 (필요 시 조정 가능)
                for attempt in range(max_attempts):
                    try:
                        self.data_cache[exchange_name][name] = future.result()
                        break  # 성공하면 반복문 탈출
                    except Exception as e:
                        print(f"Error loading {name} for {exchange_name}: {str(e)}, attempt {attempt + 1}/{max_attempts}")
                        if attempt < max_attempts - 1:  # 마지막 시도가 아니면 대기
                            print(f"Retrying {name} for {exchange_name} in 7 seconds...")
                            time.sleep(7)  # 대기
                        else:
                            print(f"Failed to load {name} for {exchange_name} after {max_attempts} attempts")

        markets = self.data_cache[exchange_name]['markets']
        self.data_cache[exchange_name]['active_tickers'] = markets.loc[:, 
            (markets.loc['swap'] == True) &
            (markets.loc['active'] == True) &
            (markets.loc['settle'].isin(['USDT', 'USDC']))
        ].columns

        return None
    
    def _update_cache(self, exchange_name, keys_to_update=None):
        """특정 캐시 데이터를 업데이트하는 헬퍼 함수"""
        exchange = self.exchanges[exchange_name]
        cache = self.data_cache[exchange_name]
        
        
        if keys_to_update is None or 'funding_rates' in keys_to_update:
            cache['funding_rates'] = pd.DataFrame(exchange.fetch_funding_rates())
            print(f"[Debug complete] {exchange_name} funding_rates cache updated")
        
        if keys_to_update is None or 'markets' in keys_to_update:
            if exchange_name == 'gateio':
                #precision 정보를 추가하기 위해 load_markets를 수정
                ##info에서 quanto_multiplier를 가져오기
                gateio_load_markets=pd.DataFrame(exchange.load_markets())
                gateio_load_markets_info=gateio_load_markets.loc['info'].copy()
                gateio_quanto_multiplier=gateio_load_markets_info.apply(pd.Series)['quanto_multiplier'].dropna().astype('float64')

                ##precision 정보에 quanto_multiplier를 추가
                ###dict 변환
                gateio_load_markets_precision_df=gateio_load_markets.loc['precision'].apply(pd.Series)
                gateio_load_markets_precision_df_amount_series=gateio_load_markets_precision_df['amount'].copy()
                gateio_load_markets_precision_df_amount_series[gateio_quanto_multiplier.index]=gateio_quanto_multiplier
                gateio_load_markets_precision_df['amount']=gateio_load_markets_precision_df_amount_series
                gateio_precision_dict=gateio_load_markets_precision_df.T.to_dict()

                ##precision 정보에 quanto_multiplier를 추가
                gateio_load_markets.loc['precision']=gateio_precision_dict
                cache['markets'] = gateio_load_markets
                print(f"[Debug complete] {exchange_name} markets and active_tickers cache updated")
            else:
                loaded_markets = pd.DataFrame(exchange.load_markets())
                cache['markets'] = loaded_markets
                cache['active_tickers'] = loaded_markets.loc[:, 
                    (loaded_markets.loc['swap'] == True) &
                    (loaded_markets.loc['active'] == True) &
                    (loaded_markets.loc['settle'].isin(['USDT', 'USDC']))
                ].columns
                print(f"[Debug complete] {exchange_name} markets and active_tickers cache updated")
        
        if keys_to_update is None or 'funding_intervals' in keys_to_update:
            cache['funding_intervals'] = pd.DataFrame(exchange.fetch_funding_intervals()).loc['interval']
            print(f"[Debug complete] {exchange_name} funding_intervals cache updated")
        
        if keys_to_update is None or 'bids_asks' in keys_to_update:
            cache['bids_asks'] = pd.DataFrame(exchange.fetchBidsAsks(params={"type": "swap", "futures": True}))
            print(f"[Debug complete] {exchange_name} bids_asks cache updated")
        
        if keys_to_update is None or 'tickers' in keys_to_update:
            cache['tickers'] = pd.DataFrame(exchange.fetchTickers(params={"type": "swap"}))
            print(f"[Debug complete] {exchange_name} tickers cache updated")

        # balance 업데이트 (추가)
        if keys_to_update is None or 'balance' in keys_to_update:
            try:
                cache['balance'] = exchange.fetch_balance()
                print(f"[Debug complete] {exchange_name} balance cache updated")
            except Exception as e:
                print(f"Failed to update balance for {exchange_name}: {str(e)}")

        if keys_to_update is None or 'balance_usdc' in keys_to_update:
            try:
                cache['balance_usdc'] = exchange.fetch_balance(params={"productType": "USDC-FUTURES"})
                print(f"[Debug complete] {exchange_name} balance_usdc cache updated")
            except Exception as e:
                print(f"Failed to update balance_usdc for {exchange_name}: {str(e)}")

        if keys_to_update is None or 'positions' in keys_to_update:
            cache['positions'] = exchange.fetch_positions()
            exchange_fetch_positions=exchange.fetch_positions()
            if len(exchange_fetch_positions)==0:
                return pd.DataFrame()
            
            elif exchange_name == 'binance':
                positions_df= pd.DataFrame(exchange.fetch_positions())
                positions_df['leverage'] = positions_df['notional']/positions_df['initialMargin']
            else:
                positions_df= pd.DataFrame(exchange.fetch_positions())

            symbol_series=positions_df['symbol']
            ticker_array, baseCoin_array, quoteCoin_array, scaling_factor=process_ticker_index(symbol_series)
            positions_df.index=baseCoin_array
            positions_df['baseCoin']=baseCoin_array
            positions_df['quoteCoin']=quoteCoin_array
            positions_df['scaling_factor']=scaling_factor
            cache['positions']= positions_df
            print(f"[Debug complete] {exchange_name} positions cache updated")



    def handling_exchange_data(self, exchange_name):
        """단일 거래소 데이터 수집 (캐시에서 티커로 슬라이싱)"""
        print(f"Handling {exchange_name} data...")
        cache = self.data_cache[exchange_name]
        ticker = cache['active_tickers']

        # funding_rates 로드 및 업데이트
        try:
            funding_rates = cache['funding_rates'][ticker].copy()
        except Exception as e:
            print(f"Failed to load funding_rates for {exchange_name}: {str(e)}, updating cache...")
            self._update_cache(exchange_name, keys_to_update=['funding_rates', 'markets','balence','positions'])
            ticker = cache['active_tickers']  # ticker 갱신 반영
            funding_rates = cache['funding_rates'][ticker].copy()

        # 거래소별 처리
        try:
            if exchange_name == 'binance':
                funding_rates.loc['interval'] = cache['funding_intervals'].reindex(ticker).fillna('8h')
                head_df = cache['bids_asks'].loc[['bid', 'ask'], ticker]
                tail_df = cache['tickers'].loc[['quoteVolume'], ticker]
                tickers = pd.concat([head_df, tail_df], axis=0)
            elif exchange_name == 'bybit':
                info = funding_rates.loc['info'].apply(pd.Series)
                tickers = info[['bid1Price', 'ask1Price', 'turnover24h']].rename(
                    columns={'bid1Price': 'bid', 'ask1Price': 'ask', 'turnover24h': 'quoteVolume'}
                ).T
            elif exchange_name == 'gateio':
                tickers = cache['tickers'][ticker].loc[['bid', 'ask', 'quoteVolume']]
                funding_rates.loc['interval'] = funding_rates.loc['interval'].replace('7200000', '2h')
            else:  # hyperliquid 등
                tickers = cache['tickers'][ticker].loc[['bid', 'ask', 'quoteVolume']]
        except Exception as e:
            print(f"Initial {exchange_name} data processing failed: {str(e)}, updating cache...")
            if exchange_name == 'binance':
                self._update_cache(exchange_name, keys_to_update=['funding_intervals', 'bids_asks', 'tickers','balence','positions'])
                funding_rates.loc['interval'] = cache['funding_intervals'].reindex(ticker).fillna('8h')
                head_df = cache['bids_asks'].loc[['bid', 'ask'], ticker]
                tail_df = cache['tickers'].loc[['quoteVolume'], ticker]
                tickers = pd.concat([head_df, tail_df], axis=0)
            elif exchange_name == 'gateio':
                self._update_cache(exchange_name, keys_to_update=['tickers','balence','positions'])
                tickers = cache['tickers'][ticker].loc[['bid', 'ask', 'quoteVolume']]
                funding_rates.loc['interval'] = funding_rates.loc['interval'].replace('7200000', '2h')
            else:  # 기본 처리
                self._update_cache(exchange_name, keys_to_update=['tickers','balence','positions'])
                tickers = cache['tickers'][ticker].loc[['bid', 'ask', 'quoteVolume']]

        # 최종 데이터프레임 생성
        try:
            essential_df = pd.concat([
                funding_rates.loc[['fundingRate', 'interval', 'markPrice', 'indexPrice']],
                tickers
            ], axis=0).T

            precision_amount=self.data_cache[exchange_name]['markets'][ticker].loc['precision'].apply(pd.Series)['amount']
            essential_df['precision_amount']=precision_amount
            return exchange_name, essential_df
        except Exception as e:
            print(f"Error finalizing {exchange_name} data: {str(e)}")
            return exchange_name, None

    def handling_bitget_data(self):
        """Bitget 데이터 처리 (오류 시 캐시 업데이트)"""
        print("Handling bitget data...")
        try:
            cache = self.data_cache['bitget']

            # 1. markets 및 ticker 데이터 로드
            try:
                markets = cache['markets']
                ticker_usdt = markets.loc[:, 
                    (markets.loc['swap'] == True) &
                    (markets.loc['active'] == True) &
                    (markets.loc['settle'] == 'USDT')
                ].columns
                ticker_usdc = markets.loc[:, 
                    (markets.loc['swap'] == True) &
                    (markets.loc['active'] == True) &
                    (markets.loc['settle'] == 'USDC')
                ].columns
            except Exception as e:
                print(f"Failed to load markets for bitget: {str(e)}, updating cache...")
                self._update_cache('bitget', keys_to_update=['markets','balence','balence_usdc','positions'])
                markets = cache['markets']
                ticker_usdt = markets.loc[:, 
                    (markets.loc['swap'] == True) &
                    (markets.loc['active'] == True) &
                    (markets.loc['settle'] == 'USDT')
                ].columns
                ticker_usdc = markets.loc[:, 
                    (markets.loc['swap'] == True) &
                    (markets.loc['active'] == True) &
                    (markets.loc['settle'] == 'USDC')
                ].columns

            # 2. funding_rates 및 funding_rates_usdc 로드
            try:
                funding_usdt = cache['funding_rates'][ticker_usdt]
                funding_usdc = cache['funding_rates_usdc'][ticker_usdc]
                funding_rates = pd.concat([funding_usdt, funding_usdc], axis=1)
            except Exception as e:
                print(f"Failed to load funding rates for bitget: {str(e)}, updating cache...")
                self._update_cache('bitget', keys_to_update=['funding_rates', 'funding_rates_usdc','balence','balence_usdc','positions'])
                funding_usdt = cache['funding_rates'][ticker_usdt]
                funding_usdc = cache['funding_rates_usdc'][ticker_usdc]
                funding_rates = pd.concat([funding_usdt, funding_usdc], axis=1)


            funding_usdt = cache['funding_rates'][ticker_usdt]
            funding_usdc = cache['funding_rates_usdc'][ticker_usdc]
            funding_rates = pd.concat([funding_usdt, funding_usdc], axis=1)
            
            info = funding_rates.loc['info'].apply(pd.Series)
            tickers = info[['bidPr', 'askPr', 'quoteVolume']]
            tickers.columns = ['bid', 'ask', 'quoteVolume']

            # self.utc_now를 사용한 시간 기반 파일명 생성
            rounded_time = self.utc_now.replace(minute=0, second=0, microsecond=0)
            time_str = rounded_time.strftime("%Y-%m-%d-%H-00")
            pkl_file = f'fundingtime_df_{time_str}.pkl'

            # .pkl 파일 읽기 시도
            fetch_new_data = False
            try:
                funding_time_df = pd.read_pickle(pkl_file)
                print(f"    Loaded funding time data from {pkl_file}")
                
                # 이미 파싱된 상태로 가정하므로 추가 파싱 생략
                # 조건 체크: ratePeriod와 funding_rates의 interval 개수 비교
                past_count=funding_time_df['ratePeriod'].count()
                current_len=len(funding_rates.loc['interval'])
                if past_count != current_len:
                    print(f"     Mismatch in counts detected,past count:{past_count},current len:{current_len} fetching new funding time data...")
                    fetch_new_data = True

            except FileNotFoundError:
                print(f"    {pkl_file} not found, fetching new funding time data...")
                fetch_new_data = True
            except Exception as e:
                print(f"    Error loading {pkl_file}: {str(e)}, fetching new funding time data...")
                fetch_new_data = True

            # 새 데이터 가져오기
            if fetch_new_data:
                funding_time_df = self.fetch_bitget_funding_times()
                # .pkl 파일로 저장 (파싱된 상태로 저장됨)
                funding_time_df.to_pickle(pkl_file)
                print(f"    Saved new funding time data to {pkl_file}")

            # 인덱스 매핑 및 재정렬
            ccxtTicker_bitgetTicker_MappingSeries = info['symbol']
            bitgetTicker_ccxtTicker_MappingSeries = pd.Series(
                ccxtTicker_bitgetTicker_MappingSeries.index,
                index=ccxtTicker_bitgetTicker_MappingSeries.values
            )
            funding_time_df.index = bitgetTicker_ccxtTicker_MappingSeries.reindex(funding_time_df['symbol']).values
            funding_time_df = funding_time_df.reindex(funding_rates.columns)

            # interval 설정
            funding_rates.loc['interval'] = funding_time_df['ratePeriod'].astype('int32')

            # 최종 데이터프레임 생성
            essential_df = pd.concat([
                funding_rates.loc[['fundingRate', 'interval', 'markPrice', 'indexPrice']].T,
                tickers
            ], axis=1)

            precision_amount=self.data_cache['bitget']['markets'][essential_df.index].loc['precision'].apply(pd.Series)['amount']
            essential_df['precision_amount']=precision_amount
            return 'bitget', essential_df

        except Exception as e:
            print(f"Error in handling_bitget_data function: {str(e)}")
            return 'bitget', None
        

    def fetch_bitget_funding_times(self):
        #기존 pkl 파일 삭제
        
        def delete_pkl_files(source_code_name='ccxt_arb_SK.ipynb'):
            print(f"현재 {source_code_name} 파일이 있는 디렉토리의 모든 pickle 파일 삭제")
            # 현재 Python 파일이 있는 디렉토리 경로의 pickle 삭제하기
            current_dir = os.path.dirname(os.path.abspath(source_code_name))

            # 디렉토리 내 모든 파일 탐색
            for filename in os.listdir(current_dir):
                # 파일이 .pkl 확장자를 가졌는지 확인
                if filename.endswith('.pkl'):
                    file_path = os.path.join(current_dir, filename)
                    try:
                        os.remove(file_path)
                        print(f"삭제됨: {filename}")
                    except Exception as e:
                        print(f"삭제 실패: {filename} - 오류: {e}")
        delete_pkl_files()
        """Bitget 펀딩 시간 API 호출 (429 에러 시 재시도)"""
        url = "https://api.bitget.com/api/v2/mix/market/funding-time"
        ticker = self.data_cache['bitget']['active_tickers']

        is_usdt = ticker.str.contains('USDT:USDT')
        is_usdc = ticker.str.contains('USDC:USDC')
        symbols_for_bitget = self.data_cache['bitget']['markets'].loc['id', ticker].values

        product_types = np.where(is_usdt, 'usdt-futures', 
                                np.where(is_usdc, 'usdc-futures', None))

        symbols = list(zip(symbols_for_bitget, product_types))

        funding_data = []

        def fetch_funding_time_api(symbol, product_type, max_retries=3):
            if product_type is None:
                return None
            params = {"symbol": symbol, "productType": product_type}
            for attempt in range(max_retries):
                try:
                    response = requests.get(url, params=params)
                    if response.status_code == 200:
                        return {"symbol": symbol, "productType": product_type, "data": response.json().get("data", [])}
                    elif response.status_code == 429:  # Too Many Requests
                        print(f"❌ {symbol} 요청 실패: Too Many Requests (429), {attempt + 1}/{max_retries} 재시도 중...")
                        time.sleep(0.1)  # 0.1초 대기
                    else:
                        print(f"❌ {symbol} Funding Time 요청 실패: {response.status_code}, {response.text}")
                        return None
                except Exception as e:
                    print(f"❌ {symbol} 요청 중 오류 발생: {str(e)}, {attempt + 1}/{max_retries} 재시도 중...")
                    time.sleep(0.1)  # 네트워크 오류 시에도 대기
            print(f"❌ {symbol} 요청 최대 재시도 초과")
            return None

        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(fetch_funding_time_api, s, p) for s, p in symbols]
            with tqdm(total=len(futures), desc="Fetching Bitget Funding Times") as pbar:
                for future in as_completed(futures):
                    result = future.result()
                    if result:
                        funding_data.append(result)
                    pbar.update(1)
        
        funding_df = pd.DataFrame(funding_data)
        if not funding_df.empty:
            funding_df = funding_df['data'].apply(pd.Series)[0].apply(pd.Series)
        else:
            funding_df = pd.DataFrame()
        return funding_df
    
    def process_all_exchanges(self):
        # 위에서 수정한 process_all_exchanges 메서드
        results = {}
        for name in self.exchanges_to_fetch:
            if name == 'bitget':
                results[name] = self.handling_bitget_data()[1]
            else:
                results[name] = self.handling_exchange_data(name)[1]
        
        FundingRate_Whole = pd.concat(
            [results[name] for name in self.exchanges_to_fetch], 
            axis=0, 
            keys=self.exchanges_to_fetch
        )

        ticker_index = FundingRate_Whole.index.get_level_values(1)
        ticker, base_coin, quote_coin, scaling_factor = process_ticker_index(ticker_index)
        FundingRate_Whole['ticker'] = ticker
        FundingRate_Whole['scaling_factor'] = scaling_factor
        
        FundingRate_Whole.index = [
            FundingRate_Whole.index.get_level_values(0),
            base_coin,
            quote_coin
        ]
        FundingRate_Whole['interval'] = FundingRate_Whole['interval'].astype('str').str.replace('h', '').astype('int32')
        FundingRate_Whole = FundingRate_Whole[FundingRate_Whole.index.get_level_values(2).notna()]

        try:
            exchange_Bybit = self.exchanges['bybit']
        except:
            exchange_Bybit=ccxt.bybit()
        USDC_USDT_Spot = exchange_Bybit.fetch_ticker('USDC/USDT')
        USDC_USDT_Spot_bid = USDC_USDT_Spot['bid']
        USDC_USDT_Spot_ask = USDC_USDT_Spot['ask']


        FundingRate_Whole_bid = FundingRate_Whole['bid'].astype('float64')
        FundingRate_Whole['bid_USDC_scaled'] = np.where(
            FundingRate_Whole.index.get_level_values(2) == 'USDC',
            FundingRate_Whole_bid,
            FundingRate_Whole_bid * USDC_USDT_Spot_ask
        ) / FundingRate_Whole['scaling_factor']

        FundingRate_Whole_ask = FundingRate_Whole['ask'].astype('float64')
        FundingRate_Whole['ask_USDC_scaled'] = np.where(
            FundingRate_Whole.index.get_level_values(2) == 'USDC',
            FundingRate_Whole_ask,
            FundingRate_Whole_ask * USDC_USDT_Spot_ask
        ) / FundingRate_Whole['scaling_factor']

        FundingRate_Whole.index.names = ['exchange', 'base_coin', 'quote_coin']
        FundingRate_Whole.index = FundingRate_Whole.index.reorder_levels(['base_coin', 'quote_coin', 'exchange'])
        FundingRate_Whole.sort_index(inplace=True)

        # maker_fee와 taker_fee를 배열로 한 번에 추가
        exchanges = FundingRate_Whole.index.get_level_values(2)
        maker_fee = np.full(len(exchanges), 0.0002, dtype=float)
        taker_fee = np.full(len(exchanges), 0.0005, dtype=float)

        # 거래소별 수수료 설정
        maker_fee[exchanges == 'gateio'] = 0.0002 * 0.6
        maker_fee[exchanges == 'hyperliquid'] = 0.000096
        maker_fee[exchanges == 'bitget'] = 0.00018
        maker_fee[exchanges == 'binance'] = 0.00018

        taker_fee[exchanges == 'gateio'] = 0.00075 * 0.6
        taker_fee[exchanges == 'hyperliquid'] = 0.000336
        taker_fee[exchanges == 'bitget'] = 0.0004
        taker_fee[exchanges == 'binance'] = 0.00045

        FundingRate_Whole['maker_fee'] = maker_fee
        FundingRate_Whole['taker_fee'] = taker_fee

        FundingRate_Whole['quoteVolume'] = FundingRate_Whole['quoteVolume'].astype('float64').round(0).astype('int64')
        
        try:
            FundingRate_Whole.drop([('NEIRO', 'USDT', 'gateio')], inplace=True)
        except:
            pass
        FundingRate_Whole = FundingRate_Whole.rename(index={"NEIRO": "NEIROCTO"}, level=0)

        CashFlow_Freq=8/FundingRate_Whole['interval']
        FundingRate_Whole['fundingRate_8H']=(FundingRate_Whole['fundingRate']*CashFlow_Freq)

        FundingRate_Whole['precisionAmount_DecimalPlace'] = np.log10(FundingRate_Whole['precision_amount']).abs().astype('int32')
        FundingRate_Whole.drop('precision_amount', axis=1, inplace=True)

        return FundingRate_Whole


    # 추가된 메서드: get_exchange_balances
    def get_exchange_balances_N_positions(self,target_leverage=3):
        """각 거래소의 잔액 데이터와 EXCHANGE_COINS에 맞춘 notional을 가져와 통합된 DataFrame으로 반환"""
        # 거래소별 지원 코인 정의
        EXCHANGE_COINS = {
            'bitget': {'balance_usdc': 'USDC', 'balance': 'USDT'},
            'hyperliquid': {'balance': 'USDC'},
            'binance': {'balance': ['USDC', 'USDT', 'BNB']},
            'gateio': {'balance': 'USDT'},
            'bybit': {'balance': ['USDT', 'USDC']}
        }

        exchange_balance_whole_list = []
        notional_data = {}  # MultiIndex를 위한 딕셔너리

        for exchange_name in self.exchanges_to_fetch:
            print(f"Processing {exchange_name} balance data...")
            coin_config = EXCHANGE_COINS.get(exchange_name, {})
            balances = []

            # 잔액 데이터 수집
            for cache_key, coins in coin_config.items():
                if isinstance(coins, str):
                    coins = [coins]
                    
                for coin in coins:
                    try:
                        balance = pd.Series(self.data_cache[exchange_name][cache_key][coin])
                        balances.append(balance)
                    except KeyError:
                        print(f"Warning: No {coin} balance data for {exchange_name}")
                        balances.append(pd.Series(index=['free', 'total'], data=[0, 0], name=coin))

            # 모든 코인 데이터를 한 번에 결합
            flat_coins = [coin for coins in coin_config.values() for coin in (coins if isinstance(coins, list) else [coins])]
            exchange_balance_whole = pd.concat(balances, axis=1, keys=flat_coins)
            exchange_balance_whole_list.append(exchange_balance_whole)

            # EXCHANGE_COINS에 정의된 코인에 대해서만 notional 계산
            try:
                print(f"Processing {exchange_name} notional data...")
                positions_df = self.data_cache[exchange_name]['positions']
                if len(positions_df) == 0:
                    print(f"No positions found for {exchange_name}")
                    for coin in flat_coins:
                        notional_data[(exchange_name, coin)] = 0
                else:
                    for coin in flat_coins:
                        notional = positions_df.loc[positions_df['quoteCoin'] == coin, 'notional'].sum()
                        notional_data[(exchange_name, coin)] = notional
                        print(f"{exchange_name} - {coin} notional: {notional}")
            except KeyError:
                print(f"Warning: No position data available for {exchange_name}")
                for coin in flat_coins:
                    notional_data[(exchange_name, coin)] = 0

        # 잔액 데이터를 통합
        result = pd.concat(exchange_balance_whole_list, axis=1, keys=self.exchanges_to_fetch)
        result.loc['used'] = result.loc['total'] - result.loc['free']
        result = (np.floor(result * 100) / 100).loc[['free', 'used', 'total']]
        result.index = ['free_cash[QuoteCoin]', 'used_margin[QuoteCoin]', 'total_equity[QuoteCoin]']

        # Notional 데이터를 MultiIndex DataFrame으로 변환
        notional_df = pd.DataFrame(
            [notional_data], 
            index=['notional[QuoteCoin]'], 
            columns=pd.MultiIndex.from_tuples(notional_data.keys())
        )

        # 잔액 데이터와 notional 데이터를 결합
        final_result = pd.concat([result, notional_df], axis=0)

        # target_Maintanance_Margin 계산
        target_Maintanance_Margin=(final_result.loc['notional[QuoteCoin]']).copy() /target_leverage
        target_Maintanance_Margin[('binance','BNB')]=0.01
        final_result.loc['target_Maintanance_Margin[QuoteCoin]'] = target_Maintanance_Margin
        final_result.loc['excess_equity[QuoteCoin]']=final_result.loc['total_equity[QuoteCoin]']-final_result.loc['target_Maintanance_Margin[QuoteCoin]']

        order_available_cash=final_result.loc[['free_cash[QuoteCoin]','excess_equity[QuoteCoin]']].min()
        order_available_cash[order_available_cash<0]=0
        final_result.loc['order_available_cash[QuoteCoin]']=order_available_cash

        try:
            USDC_USDT_Bid_Ask=self.exchanges['bybit'].fetch_ticker('USDC/USDT')
        except:
            USDC_USDT_Bid_Ask=ccxt.bybit().fetch_ticker('USDC/USDT')
        USDC_USDT_Ask=USDC_USDT_Bid_Ask['ask']
        USDC_USDT_Bid=USDC_USDT_Bid_Ask['bid']

        try:
            BNB_USDC_Bid=self.exchanges['binance'].fetch_ticker('BNB/USDC')['bid']
            BNB_USDT_Bid=self.exchanges['binance'].fetch_ticker('BNB/USDT')['bid']
        except:
            BNB_USDC_Bid=ccxt.binance().fetch_ticker('BNB/USDC')['bid']
            BNB_USDT_Bid=ccxt.binance().fetch_ticker('BNB/USDT')['bid']

        quote_coin_price_USDC=np.array(final_result.columns.get_level_values(1))
        quote_coin_price_USDC[quote_coin_price_USDC=='USDT']=USDC_USDT_Ask #buy USDC/USDT
        quote_coin_price_USDC[quote_coin_price_USDC=='USDC']=1
        quote_coin_price_USDC[quote_coin_price_USDC=='BNB']=BNB_USDC_Bid   #Sell BNB/USDC
        final_result.loc['total_equity[USDC]']=(final_result.loc['total_equity[QuoteCoin]']*quote_coin_price_USDC)

        quote_coin_price_USDT=np.array(final_result.columns.get_level_values(1))
        quote_coin_price_USDT[quote_coin_price_USDT=='USDT']=1
        quote_coin_price_USDT[quote_coin_price_USDT=='USDC']=1/USDC_USDT_Bid #Sell USDC/USDT
        quote_coin_price_USDT[quote_coin_price_USDT=='BNB']=BNB_USDT_Bid
        final_result.loc['total_equity[USDT]']=(final_result.loc['total_equity[QuoteCoin]']*quote_coin_price_USDT)

        final_result=(np.floor(final_result * 100) / 100)

        final_result.loc['order_available_cash[USDC]']=final_result.loc['order_available_cash[QuoteCoin]']*quote_coin_price_USDC

        self.magin_dashboards=final_result
        print("magin_dashboards updated")
        return final_result

# Analyzer Class

In [93]:
class FundingRateAnalyzer:
    def __init__(self, fetcher):
        self.fetcher = fetcher
        self.FundingRate_Whole = None
        self.funding_frame_df = None
        self.opt_decision_sorted=None

    def fetch_data(self):
        """거래소 데이터를 가져와 FundingRate_Whole을 초기화"""
        self.FundingRate_Whole = self.fetcher.process_all_exchanges()

        if len(self.fetcher.exchanges_to_fetch) != len(self.FundingRate_Whole.index.get_level_values(2).unique()):
            print("Error: Not all exchanges' data fetched correctly - retrying...")
            self.fetcher=ExchangeDataFetcher(self.fetcher.exchanges_to_fetch)
            self.FundingRate_Whole = self.fetcher.process_all_exchanges()
            print(f"Retry complete, candiate number:{len(self.fetcher.exchanges_to_fetch)},fetched exchanges:{self.FundingRate_Whole.index.get_level_values(2).unique()}")
        print("Data fetched successfully, FundingRate_Whole initialized")
        return self.FundingRate_Whole

    def get_next_n_hours_intervals(self, utc_now, n):
        """현재 시간 이후 N시간 동안의 정각 시간대를 반환"""
        next_hour = (utc_now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
        return pd.date_range(start=next_hour, periods=n, freq='h')

    def create_funding_frame(self, horizon=8):
        """FundingRate_Whole을 기반으로 funding_frame_df 생성"""
        time_index = self.get_next_n_hours_intervals(self.fetcher.utc_now, horizon)
        funding_frame_list = []

        for interval in self.FundingRate_Whole['interval'].unique():
            funding_count = int(horizon / interval)
            filtered_intervals = time_index[time_index.hour % interval == 0][:funding_count]
            funding_frame_df_per_interval = pd.DataFrame(
                [self.FundingRate_Whole.loc[self.FundingRate_Whole['interval'] == interval, 'fundingRate']] * funding_count,
                index=filtered_intervals
            )
            funding_frame_list.append(funding_frame_df_per_interval)

        funding_frame_df = pd.concat(funding_frame_list, axis=1).fillna(0)
        funding_frame_df.sort_index(inplace=True)
        return funding_frame_df

    def calculate_base_differences(self, funding_frame_df):
        """base 코인을 기준으로 컬럼 간 차이를 MultiIndex 컬럼으로 계산 (벡터화)"""
        diff_data = []
        column_pairs = []
        grouped = funding_frame_df.T.groupby(level=0)

        for base, df in grouped:
            cols = df.index
            values = df.values.T
            diff = values[:, :, np.newaxis] - values[:, np.newaxis, :]
            col_pairs = list(permutations(cols, 2))
            for col1, col2 in col_pairs:
                long_leg = (base, col1[1], col1[2])
                short_leg = (base, col2[1], col2[2])
                diff_data.append(diff[:, cols.get_loc(col2), cols.get_loc(col1)])
                column_pairs.append((long_leg, short_leg))

        diff_array = np.stack(diff_data, axis=1)
        return pd.DataFrame(
            diff_array,
            index=funding_frame_df.index,
            columns=pd.MultiIndex.from_tuples(column_pairs, names=["Long Leg", "Short Leg"])
        )

    def filter_positive_return_pairs(self, funding_spread_df):
        """포지션을 잡고 다음 시간 funding rate 수취가 음수일 때 포지션을 감는 전략이 양수인 쌍만 필터링"""
            #포지션을 잡고
        expected_return_positive_df=funding_spread_df[(funding_spread_df>=0)]

        def fill_nan_afterwards(df):
            """NaN을 만나면 그 밑을 NaN으로 모두 채우는 함수"""
            mask = df.isna().cumsum() > 0
            df[mask] = np.nan
            return df
        
            #다음시간 funding rate 수취가 음수일 때 이후는 NaN으로 채움
        expected_return_positive_df_FillNan=fill_nan_afterwards(expected_return_positive_df)
            # 그 합산이 양수인 것만 필터링
        positive_return_pairs =expected_return_positive_df_FillNan.loc[:,expected_return_positive_df_FillNan.sum().round(6)>0]

        return positive_return_pairs



    def calculate_price_impact_and_fees(self, filtered_df):
        """가격 영향과 수수료 계산"""
        long_leg_bid = self.FundingRate_Whole.loc[filtered_df.columns.get_level_values(0), 'bid_USDC_scaled'].values
        long_leg_ask = self.FundingRate_Whole.loc[filtered_df.columns.get_level_values(0), 'ask_USDC_scaled'].values
        short_leg_bid = self.FundingRate_Whole.loc[filtered_df.columns.get_level_values(1), 'bid_USDC_scaled'].values
        short_leg_ask = self.FundingRate_Whole.loc[filtered_df.columns.get_level_values(1), 'ask_USDC_scaled'].values

        long_maker_short_taker_pi = pd.Series((short_leg_bid - long_leg_bid) / long_leg_bid, index=filtered_df.columns)
        short_maker_long_taker_pi = pd.Series((short_leg_ask - long_leg_ask) / long_leg_ask, index=filtered_df.columns)

        long_leg_bid_fee = self.FundingRate_Whole.loc[filtered_df.columns.get_level_values(0), 'maker_fee'].values
        long_leg_ask_fee = self.FundingRate_Whole.loc[filtered_df.columns.get_level_values(0), 'taker_fee'].values
        short_leg_bid_fee = self.FundingRate_Whole.loc[filtered_df.columns.get_level_values(1), 'maker_fee'].values
        short_leg_ask_fee = self.FundingRate_Whole.loc[filtered_df.columns.get_level_values(1), 'taker_fee'].values

        long_maker_short_taker_fee = pd.Series(long_leg_bid_fee + short_leg_ask_fee, index=filtered_df.columns)
        short_maker_long_taker_fee = pd.Series(short_leg_bid_fee + long_leg_ask_fee, index=filtered_df.columns)

        return {
            'long_maker_short_taker_pi': long_maker_short_taker_pi,
            'short_maker_long_taker_pi': short_maker_long_taker_pi,
            'long_maker_short_taker_fee': long_maker_short_taker_fee,
            'short_maker_long_taker_fee': short_maker_long_taker_fee
        }

    def apply_fees_to_returns(self, filtered_df, price_impact_fees):
        """수익률에 수수료 적용"""
        long_maker_short_taker_df = filtered_df.copy()
        short_maker_long_taker_df = filtered_df.copy()
        long_maker_short_taker_df.iloc[0] += (price_impact_fees['long_maker_short_taker_pi'] - price_impact_fees['long_maker_short_taker_fee'])
        short_maker_long_taker_df.iloc[0] += (price_impact_fees['short_maker_long_taker_pi'] - price_impact_fees['short_maker_long_taker_fee'])

        is_next_nan = pd.isna(long_maker_short_taker_df).shift(-1).copy()
        is_next_nan.iloc[-1] = False
        long_maker_short_taker_fee_df = pd.DataFrame(
            np.where(
                is_next_nan,
                price_impact_fees['long_maker_short_taker_fee'].values,
                0
            ),
            index=long_maker_short_taker_df.index,
            columns=long_maker_short_taker_df.columns
        )

        short_maker_long_taker_fee_df = pd.DataFrame(
            np.where(
                is_next_nan,
                price_impact_fees['short_maker_long_taker_fee'].values,
                0
            ),
            index=short_maker_long_taker_df.index,
            columns=short_maker_long_taker_df.columns
        )

        long_maker_short_taker_df -= long_maker_short_taker_fee_df
        short_maker_long_taker_df -= short_maker_long_taker_fee_df

        return long_maker_short_taker_df, short_maker_long_taker_df

    def analyze(self, horizon=8):
        """전체 분석 프로세스 실행"""
        funding_frame_df = self.create_funding_frame(horizon)
        print("Funding frame created successfully horizon=",horizon)
        self.funding_frame_df=funding_frame_df
        
        funding_spread_df = self.calculate_base_differences(funding_frame_df)
        #포지션을 잡고 funding rate 수취 양수 수익률 쌍만 필터링
        positive_fill_nan_df = self.filter_positive_return_pairs(funding_spread_df)
        #두 페어 모두 8H Funding Rate 합산이 1BP인 페어 negative screening
        longLeg_1BP=(self.FundingRate_Whole['fundingRate_8H'][positive_fill_nan_df.columns.get_level_values(0)]==0.0001).values
        shortLeg_1BP=(self.FundingRate_Whole['fundingRate_8H'][positive_fill_nan_df.columns.get_level_values(1)]==0.0001).values
        positive_fill_nan_df= positive_fill_nan_df.loc[:,(~longLeg_1BP)|(~shortLeg_1BP)]
        #다음 시 정각 funding rate 수취 양수 수익률 쌍만 필터링
        filtered_df = positive_fill_nan_df.loc[:,positive_fill_nan_df.iloc[0] > 0]
        price_impact_fees = self.calculate_price_impact_and_fees(filtered_df)
        long_maker_short_taker_df, short_maker_long_taker_df = self.apply_fees_to_returns(filtered_df, price_impact_fees)
        #Positive PI를 제외하고, 수수료를 내면서 포지션을 잡고 funding rate 수취 양수 수익률 쌍말 필터링
        long_maker_short_taker_PositivePI=price_impact_fees['long_maker_short_taker_pi'].copy()
        long_maker_short_taker_PositivePI[long_maker_short_taker_PositivePI<0]=0
        short_maker_long_taker_PositivePI=price_impact_fees['short_maker_long_taker_pi'].copy()
        short_maker_long_taker_PositivePI[short_maker_long_taker_PositivePI<0]=0

        long_maker_short_taker_df_withoutPositivePI=long_maker_short_taker_df.copy()
        short_maker_long_taker_df_withoutPositivePI=short_maker_long_taker_df.copy()
        long_maker_short_taker_df_withoutPositivePI.iloc[0] = long_maker_short_taker_df_withoutPositivePI.iloc[0] - (long_maker_short_taker_PositivePI)
        short_maker_long_taker_df_withoutPositivePI.iloc[0] = short_maker_long_taker_df_withoutPositivePI.iloc[0] - (short_maker_long_taker_PositivePI)

        long_maker_short_taker_screened=long_maker_short_taker_df_withoutPositivePI.sum().round(6)>0
        long_maker_short_taker_screened_PriceImpantImplied=long_maker_short_taker_df.loc[:,long_maker_short_taker_screened]
        short_maker_long_taker_screened=short_maker_long_taker_df_withoutPositivePI.sum().round(6)>0
        short_maker_long_taker_screened_PriceImpantImplied=short_maker_long_taker_df.loc[:,short_maker_long_taker_screened]

        opt_decision = pd.concat(
            [long_maker_short_taker_screened_PriceImpantImplied, short_maker_long_taker_screened_PriceImpantImplied],
            axis=1,
            keys=['LongMaker_ShortTaker', 'ShortMaker_LongTaker']
        )
        opt_decision_sorted = opt_decision[opt_decision.sum().sort_values(ascending=False).index]
        
        
        self.opt_decision_sorted=opt_decision_sorted
        print("Analysis completed successfully, opt_decision_sorted initialized")
        return funding_frame_df, opt_decision_sorted
    
    def calculate_cost_analysis(self, opt_decision_sorted):
        """Calculate cost analysis including price impact and fees for given trading pairs"""
        cost_df = pd.DataFrame(columns=opt_decision_sorted.columns)

        # Extract bid/ask prices and fees for Long and Short legs
        bid_LongLeg = self.FundingRate_Whole.loc[cost_df.columns.get_level_values(1), 'bid_USDC_scaled']
        ask_LongLeg = self.FundingRate_Whole.loc[cost_df.columns.get_level_values(1), 'ask_USDC_scaled']
        maker_fee_LongLeg = self.FundingRate_Whole.loc[cost_df.columns.get_level_values(1), 'maker_fee']
        taker_fee_LongLeg = self.FundingRate_Whole.loc[cost_df.columns.get_level_values(1), 'taker_fee']

        bid_ShortLeg = self.FundingRate_Whole.loc[cost_df.columns.get_level_values(2), 'bid_USDC_scaled']
        ask_ShortLeg = self.FundingRate_Whole.loc[cost_df.columns.get_level_values(2), 'ask_USDC_scaled']
        maker_fee_ShortLeg = self.FundingRate_Whole.loc[cost_df.columns.get_level_values(2), 'maker_fee']
        taker_fee_ShortLeg = self.FundingRate_Whole.loc[cost_df.columns.get_level_values(2), 'taker_fee']

        # Determine whether it's LongMaker_ShortTaker strategy
        is_long_maker_short_taker = cost_df.columns.get_level_values(0) == 'LongMaker_ShortTaker'

        # Calculate prices and fees based on strategy
        price_LongLeg = np.where(is_long_maker_short_taker, bid_LongLeg, ask_LongLeg)
        price_ShortLeg = np.where(is_long_maker_short_taker, bid_ShortLeg, ask_ShortLeg)
        fee_LongLeg = -np.where(is_long_maker_short_taker, maker_fee_LongLeg, taker_fee_LongLeg)
        fee_ShortLeg = -np.where(is_long_maker_short_taker, taker_fee_ShortLeg, maker_fee_ShortLeg)

        # Populate cost_df with calculated values
        cost_df.loc['Price_LongLeg'] = price_LongLeg
        cost_df.loc['Price_ShortLeg'] = price_ShortLeg
        cost_df.loc['Price_Impact'] = price_ShortLeg / price_LongLeg - 1
        cost_df.loc['Fee_LongLeg'] = fee_LongLeg
        cost_df.loc['Fee_ShortLeg'] = fee_ShortLeg

        return cost_df

In [94]:
if __name__ == "__main__":
    exchanges_to_fetch=['bitget','hyperliquid','gateio','binance','bybit']
    fetcher = ExchangeDataFetcher(exchanges_to_fetch)
    analyzer = FundingRateAnalyzer(fetcher)
    analyzer.fetch_data()
    print('Funding Rate Whole')
    FundingRate_Whole=analyzer.FundingRate_Whole
    display(analyzer.FundingRate_Whole)
    analyzer.analyze(horizon=8)
    print('Funding Frame')
    display(analyzer.funding_frame_df)
    print('Opt Decision Sorted')
    display(analyzer.opt_decision_sorted)

    cost_df = analyzer.calculate_cost_analysis(analyzer.opt_decision_sorted)
    print('Cost Analysis:')
    display(cost_df)
    print('Balance:')
    fetcher.get_exchange_balances_N_positions()
    display(fetcher.magin_dashboards)
    print('utc now: ', fetcher.utc_now)

Preloading Exchanges:   0%|          | 0/5 [00:00<?, ?it/s]

Error loading funding_rates for bitget: bitget {"code":"429","msg":"Too Many Requests","requestTime":1743595703857,"data":null}, attempt 1/3
Retrying funding_rates for bitget in 7 seconds...


Preloading Exchanges:  20%|██        | 1/5 [00:06<00:24,  6.17s/it]

✅ hyperliquid 데이터 로드 완료
Error loading funding_rates for bitget: bitget {"code":"429","msg":"Too Many Requests","requestTime":1743595703857,"data":null}, attempt 2/3
Retrying funding_rates for bitget in 7 seconds...


Preloading Exchanges:  40%|████      | 2/5 [00:10<00:15,  5.25s/it]

...

Cost Analysis:


LongMaker_ShortTaker     ShortMaker_LongTaker  \
Long Leg             (DOT, USDC, bybit)       (DOT, USDC, bybit)   
Short Leg      (DOT, USDC, hyperliquid) (DOT, USDC, hyperliquid)   
Price_LongLeg                  4.030000                 4.037000   
Price_ShortLeg                 4.049430                 4.053050   
Price_Impact                   0.004821                 0.003976   
Fee_LongLeg                   -0.000200                -0.000500   
Fee_ShortLeg                  -0.000336                -0.000096   

                      LongMaker_ShortTaker        ShortMaker_LongTaker  \
Long Leg       (POPCAT, USDC, hyperliquid) (POPCAT, USDC, hyperliquid)   
Short Leg           (POPCAT, USDT, bitget)      (POPCAT, USDT, bitget)   
Price_LongLeg                     0.181949                    0.182048   
Price_ShortLeg                    0.182518                    0.182618   
Price_Impact                      0.003129                    0.003132   
Fee_LongLeg                      -0.000096                   -0.000336   
Fee_ShortLeg                     -0.000400                   -0.000180   

                                            LongMaker_ShortTaker  \
Long Leg           (MEW, USDT, binance)     (MEW, USDT, binance)   
Short Leg      (MEW, USDC, hyperliquid) (MEW, USDC, hyperliquid)   
Price_LongLeg                  0.003055                 0.003054   
Price_ShortLeg                 0.003066                 0.003063   
Price_Impact                   0.003500                 0.002847   
Fee_LongLeg                   -0.000450                -0.000180   
Fee_ShortLeg                  -0.000096                -0.000336   

                    ShortMaker_LongTaker                           \
Long Leg       (COMP, USDC, hyperliquid)       (ARK, USDT, bybit)   
Short Leg           (COMP, USDT, gateio) (ARK, USDC, hyperliquid)   
Price_LongLeg                  42.942900                 0.292229   
Price_ShortLeg                 43.044304                 0.292731   
Price_Impact                    0.002361                 0.001717   
Fee_LongLeg                    -0.000336                -0.000500   
Fee_ShortLeg                   -0.000120                -0.000096   

                                                                       ...  \
Long Leg            (CHILLGUY, USDT, gateio)      (BIO, USDT, gateio)  ...   
Short Leg      (CHILLGUY, USDC, hyperliquid) (BIO, USDC, hyperliquid)  ...   
Price_LongLeg                       0.024562                 0.069437  ...   
Price_ShortLeg                      0.024607                 0.069515  ...   
Price_Impact                        0.001813                 0.001124  ...   
Fee_LongLeg                        -0.000450                -0.000450  ...   
Fee_ShortLeg                       -0.000096                -0.000096  ...   

                   LongMaker_ShortTaker     ShortMaker_LongTaker  \
Long Leg             (ARK, USDT, bybit) (REZ, USDC, hyperliquid)   
Short Leg      (ARK, USDC, hyperliquid)       (REZ, USDT, bybit)   
Price_LongLeg                  0.292129                 0.018227   
Price_ShortLeg                 0.292114                 0.018232   
Price_Impact                  -0.000052                 0.000265   
Fee_LongLeg                   -0.000200                -0.000336   
Fee_ShortLeg                  -0.000336                -0.000200   

                                             LongMaker_ShortTaker  \
Long Leg       (FLOKI, USDC, hyperliquid) (IP, USDC, hyperliquid)   
Short Leg           (FLOKI, USDT, bitget)      (IP, USDT, bitget)   
Price_LongLeg                    0.000057                4.134020   
Price_ShortLeg                   0.000057                4.135213   
Price_Impact                     0.000344                0.000289   
Fee_LongLeg                     -0.000336               -0.000096   
Fee_ShortLeg                    -0.000180               -0.000400   

                      ShortMaker_LongTaker     LongMaker_Sho

Balance:
Processing bitget balance data...
Processing bitget notional data...
bitget - USDC notional: 0.0
bitget - USDT notional: 29116.5146
Processing hyperliquid balance data...
Processing hyperliquid notional data...
No positions found for hyperliquid
Processing gateio balance data...
Processing gateio notional data...
gateio - USDT notional: 9628.02
Processing binance balance data...
Processing binance notional data...
binance - USDC notional: 0.0
binance - USDT notional: 16560.6
binance - BNB notional: 0.0
Processing bybit balance data...
Processing bybit notional data...
No positions found for bybit
magin_dashboards updated


bitget               hyperliquid  \
                                       USDC          USDT        USDC   
free_cash[QuoteCoin]                   5.05  19670.450000     5000.83   
used_margin[QuoteCoin]                 0.00  17285.200000        0.00   
total_equity[QuoteCoin]                5.05  36955.660000     5000.83   
notional[QuoteCoin]                    0.00  29116.510000        0.00   
target_Maintanance_Margin[QuoteCoin]   0.00   9705.500000        0.00   
excess_equity[QuoteCoin]               5.05  27250.150000     5000.83   
order_available_cash[QuoteCoin]        5.05  19670.450000     5000.83   
total_equity[USDC]                     5.05  36959.350000     5000.83   
total_equity[USDT]                     5.05  36955.660000     5000.83   
order_available_cash[USDC]             5.05  19672.417045     5000.83   

                                           gateio binance                     \
                                             USDT    USDC      USDT      BNB   
free_cash[QuoteCoin]                  29150.90000     0.0    223.70   0.0900   
used_margin[QuoteCoin]                 5398.07000     0.0   4727.55   0.0000   
total_equity[QuoteCoin]               34548.98000     0.0   4951.26   0.0900   
notional[QuoteCoin]                    9628.02000     0.0  16560.59   0.0000   
target_Maintanance_Margin[QuoteCoin]   3209.34000     0.0   5520.20   0.0100   
excess_equity[QuoteCoin]              31339.64000     0.0   -568.94   0.0800   
order_available_cash[QuoteCoin]       29150.90000     0.0      0.00   0.0800   
total_equity[USDC]                    34552.43000     0.0   4951.75  53.8800   
total_equity[USDT]                    34548.98000     0.0   4951.26  53.8800   
order_available_cash[USDC]            29153.81509     0.0      0.00  47.8952   

                                            bybit       
                                             USDT USDC  
free_cash[QuoteCoin]                  4371.950000  0.0  
used_margin[QuoteCoin]                   0.000000  0.0  
total_equity[QuoteCoin]               4371.950000  0.0  
notional[QuoteCoin]                      0.000000  0.0  
target_Maintanance_Margin[QuoteCoin]     0.000000  0.0  
excess_equity[QuoteCoin]              4371.950000  0.0  
order_available_cash[QuoteCoin]       4371.950000  0.0  
total_equity[USDC]                    4372.380000  0.0  
total_equity[USDT]                    4371.950000  0.0  
order_available_cash[USDC]            4372.387195  0.0

utc now:  2025-04-02 12:08:22.639586+00:00


In [ ]:
target_leverage = 3
initial_number_of_pairs = 5  # 초기값
number_of_pairs= initial_number_of_pairs


# Step 1: quoteVolume을 체크하여 number_of_pairs 결정

while True:
    investing_pairs = analyzer.opt_decision_sorted.columns[:number_of_pairs]
    long_leg = pd.MultiIndex.from_tuples(investing_pairs.get_level_values(1))
    short_leg = pd.MultiIndex.from_tuples(investing_pairs.get_level_values(2))
    long_leg_quoteVolume = analyzer.FundingRate_Whole.loc[long_leg,'quoteVolume'].values
    short_leg_quoteVolume = analyzer.FundingRate_Whole.loc[short_leg,'quoteVolume'].values
    quoteVolume_min = np.minimum(long_leg_quoteVolume, short_leg_quoteVolume)
    quoteVolume_Weight = np.floor(quoteVolume_min/quoteVolume_min.sum()*100)/100
    
    if (quoteVolume_Weight.round(1) > 0).sum() == initial_number_of_pairs:
        print(f"Number of pairs decided: {number_of_pairs}")
        break
    number_of_pairs += 1
    if number_of_pairs > len(analyzer.opt_decision_sorted.columns):
        print("Error: Not enough pairs to invest")
        break

investing_pairs=analyzer.opt_decision_sorted.columns[np.where(quoteVolume_Weight>0)]
long_leg=pd.MultiIndex.from_tuples(investing_pairs.get_level_values(1))
short_leg=pd.MultiIndex.from_tuples(investing_pairs.get_level_values(2))
long_leg_quoteVolume=analyzer.FundingRate_Whole.loc[long_leg,'quoteVolume'].values
short_leg_quoteVolume=analyzer.FundingRate_Whole.loc[short_leg,'quoteVolume'].values
quoteVolume_min=np.minimum(long_leg_quoteVolume, short_leg_quoteVolume)
quoteVolume_Weight=np.floor(quoteVolume_min/quoteVolume_min.sum()*100)/100
# Step 2: 결정된 number_of_pairs로 나머지 계산 수행

order_details = pd.DataFrame(columns=investing_pairs)
orderType = investing_pairs.get_level_values(0)
long_leg_exchange = long_leg.get_level_values(2)
long_leg_quoteCoin = long_leg.get_level_values(1)
short_leg_exchange = short_leg.get_level_values(2)
short_leg_quoteCoin = short_leg.get_level_values(1)

try:
    order_available_cash_USDC = fetcher.magin_dashboards.loc['order_available_cash[USDC]'].drop(['BNB'],level=1)
    order_available_cash_USDC_sum = order_available_cash_USDC.sum()
except:
    order_available_cash_USDC = fetcher.magin_dashboards.loc['order_available_cash[USDC]']
    order_available_cash_USDC_sum = order_available_cash_USDC.sum()

order_details.loc['long_leg_account'] = list(zip(long_leg_exchange, long_leg_quoteCoin))
order_details.loc['long_leg_account_order_available_cash_current[USDC]'] = order_available_cash_USDC[order_details.loc['long_leg_account'].values].values
order_details.loc['short_leg_account'] = list(zip(short_leg_exchange, short_leg_quoteCoin))
order_details.loc['short_leg_account_order_available_cash_current[USDC]'] = order_available_cash_USDC[order_details.loc['short_leg_account'].values].values

order_details.loc['long_leg_quoteVolume'] = long_leg_quoteVolume
order_details.loc['short_leg_quoteVolume'] = short_leg_quoteVolume
order_details.loc['quoteVolume_min'] = quoteVolume_min
order_details.loc['quoteVolume_Weight'] = quoteVolume_Weight

order_details.loc['allocated_margin_for_Each_pair[USDC]'] = np.floor(order_details.loc['quoteVolume_Weight']*order_available_cash_USDC_sum/2)

margin_subset = order_details.loc[['allocated_margin_for_Each_pair[USDC]','long_leg_account_order_available_cash_current[USDC]','short_leg_account_order_available_cash_current[USDC]']].copy()
order_amount_list = []
for th_pair in range(len(investing_pairs)):
    order_amount = margin_subset.iloc[:,th_pair].min()
    order_amount_list.append(order_amount)
    margin_subset_long_leg = margin_subset.loc['long_leg_account_order_available_cash_current[USDC]'].values.copy()
    margin_subset_short_leg = margin_subset.loc['short_leg_account_order_available_cash_current[USDC]'].values.copy()
    #deduction of margin from long_pair
    deduction_of_margin_From_LongPair_For_LongPair_location = (order_details.loc['long_leg_account'].iloc[th_pair]==order_details.loc['long_leg_account']).values
    deduction_of_margin_From_LongPair_For_ShortPair_location = (order_details.loc['long_leg_account'].iloc[th_pair]==order_details.loc['short_leg_account']).values
    #deduction of margin from short_pair
    deduction_of_margin_From_ShortPair_For_LongPair_location = (order_details.loc['short_leg_account'].iloc[th_pair]==order_details.loc['long_leg_account']).values
    deduction_of_margin_From_ShortPair_For_ShortPair_location = (order_details.loc['short_leg_account'].iloc[th_pair]==order_details.loc['short_leg_account']).values

    location_of_LongPair_to_change=(deduction_of_margin_From_LongPair_For_LongPair_location | deduction_of_margin_From_ShortPair_For_LongPair_location)
    location_of_ShortPair_to_change=(deduction_of_margin_From_LongPair_For_ShortPair_location | deduction_of_margin_From_ShortPair_For_ShortPair_location)
    location_of_LongPair_to_change[:th_pair+1] = False
    location_of_ShortPair_to_change[:th_pair+1] = False
    margin_subset_long_leg[location_of_LongPair_to_change] -= order_amount
    margin_subset_short_leg[location_of_ShortPair_to_change] -= order_amount
    margin_subset.loc['long_leg_account_order_available_cash_current[USDC]'] = margin_subset_long_leg
    margin_subset.loc['short_leg_account_order_available_cash_current[USDC]'] = margin_subset_short_leg

margin_subset.loc['Min[allocatedMargin,availableCash][USDC]'] = order_amount_list
margin_subset.rename(index={'long_leg_account_order_available_cash_current[USDC]':'long_leg_account_order_available_cash_BeforeOrder[USDC]'}, inplace=True)
margin_subset.rename(index={'short_leg_account_order_available_cash_current[USDC]':'short_leg_account_order_available_cash_BeforeOrder[USDC]'}, inplace=True)

order_details = pd.concat([order_details[:-1], margin_subset])

long_leg_price_USDC = np.where(orderType=='LongMaker_ShortTaker',
                            analyzer.FundingRate_Whole.loc[long_leg,'ask_USDC_scaled'],
                            analyzer.FundingRate_Whole.loc[long_leg,'bid_USDC_scaled'])
short_leg_price_USDC = np.where(orderType=='LongMaker_ShortTaker',
                            analyzer.FundingRate_Whole.loc[short_leg,'ask_USDC_scaled'],
                            analyzer.FundingRate_Whole.loc[short_leg,'bid_USDC_scaled'])
order_details.loc['long_leg_price_USDC_scaled'] = long_leg_price_USDC
order_details.loc['short_leg_price_USDC_scaled'] = short_leg_price_USDC
order_details.loc['long_leg_quantity'] = (order_details.loc['Min[allocatedMargin,availableCash][USDC]'])/long_leg_price_USDC*target_leverage
order_details.loc['short_leg_quantity'] = (order_details.loc['Min[allocatedMargin,availableCash][USDC]'])/short_leg_price_USDC*target_leverage
order_details.loc['min_quantity'] = order_details.loc[['long_leg_quantity','short_leg_quantity']].min()
order_details.loc['cash_shortage_for_Each_Pair[USDC]'] = order_details.loc['allocated_margin_for_Each_pair[USDC]']-order_details.loc['Min[allocatedMargin,availableCash][USDC]']
order_details.loc['long_leg_cash_shortage'] = order_details.loc['long_leg_account_order_available_cash_BeforeOrder[USDC]']<order_details.loc['allocated_margin_for_Each_pair[USDC]']
order_details.loc['short_leg_cash_shortage'] = order_details.loc['short_leg_account_order_available_cash_BeforeOrder[USDC]']<order_details.loc['allocated_margin_for_Each_pair[USDC]']
order_details.loc['precisionAmount_DecimalPlace_Max']= np.minimum(analyzer.FundingRate_Whole.loc[long_leg,'precisionAmount_DecimalPlace'].values,
                                                                analyzer.FundingRate_Whole.loc[short_leg,'precisionAmount_DecimalPlace'].values)
multiplier = 10 ** order_details.loc['precisionAmount_DecimalPlace_Max']
order_details.loc['min_quantity_precision_floored'] = np.floor(order_details.loc['min_quantity'] * multiplier) / multiplier
order_details

Number of pairs decided: 7


LongMaker_ShortTaker  \
Long Leg                                           (POPCAT, USDC, hyperliquid)   
Short Leg                                               (POPCAT, USDT, bitget)   
long_leg_account                                           (hyperliquid, USDC)   
long_leg_account_order_available_cash_current[U...                     5000.83   
short_leg_account                                               (bitget, USDT)   
short_leg_account_order_available_cash_current[...                19672.417045   
long_leg_quoteVolume                                                   8158595   
short_leg_quoteVolume                                                  6167805   
quoteVolume_min                                                        6167805   
quoteVolume_Weight                                                        0.23   
allocated_margin_for_Each_pair[USDC]                                      6693   
long_leg_account_order_available_cash_BeforeOrd...                     5000.83   
short_leg_account_order_available_cash_BeforeOr...                19672.417045   
Min[allocatedMargin,availableCash][USDC]                               5000.83   
long_leg_price_USDC_scaled                                            0.182048   
short_leg_price_USDC_scaled                                           0.182618   
long_leg_quantity                                                 82409.529355   
short_leg_quantity                                                82152.190038   
min_quantity                                                      82152.190038   
cash_shortage_for_Each_Pair[USDC]                                      1692.17   
long_leg_cash_shortage                                                    True   
short_leg_cash_shortage                                                  False   
precisionAmount_DecimalPlace_Max                                             0   
min_quantity_precision_floored                                         82152.0   

                                                          ShortMaker_LongTaker  \
Long Leg                                           (POPCAT, USDC, hyperliquid)   
Short Leg                                               (POPCAT, USDT, bitget)   
long_leg_account                                           (hyperliquid, USDC)   
long_leg_account_order_available_cash_current[U...                     5000.83   
short_leg_account                                               (bitget, USDT)   
short_leg_account_order_available_cash_current[...                19672.417045   
long_leg_quoteVolume                                                   8158595   
short_leg_quoteVolume                                                  6167805   
quoteVolume_min                                                        6167805   
quoteVolume_Weight                                                        0.23   
allocated_margin_for_Each_pair[USDC]                                      6693   
long_leg_account_order_available_cash_BeforeOrd...                         0.0   
short_leg_account_order_available_cash_BeforeOr...                14671.587045   
Min[allocatedMargin,availableCash][USDC]                                   0.0   
long_leg_price_USDC_scaled                                            0.181949   
short_leg_price_USDC_scaled                                           0.182518   
long_leg_quantity                                                          0.0   
short_leg_quantity                                                         0.0   
min_quantity                                                               0.0   
cash_shortage_for_Each_Pair[USDC]                                       6693.0   
long_leg_cash_shortage                                                    True   
short_leg_cash_shortage                                                  False   
precisionAmount_DecimalPlace_Max                                             0   
min_quantity_precision_floored         

In [139]:
target_leverage = 3
initial_number_of_pairs = 5  # 초기값
number_of_pairs= initial_number_of_pairs


# Step 1: quoteVolume을 체크하여 number_of_pairs 결정

while True:
    investing_pairs = analyzer.opt_decision_sorted.columns[:number_of_pairs]
    long_leg = pd.MultiIndex.from_tuples(investing_pairs.get_level_values(1))
    short_leg = pd.MultiIndex.from_tuples(investing_pairs.get_level_values(2))
    long_leg_quoteVolume = analyzer.FundingRate_Whole.loc[long_leg,'quoteVolume'].values
    short_leg_quoteVolume = analyzer.FundingRate_Whole.loc[short_leg,'quoteVolume'].values
    quoteVolume_min = np.minimum(long_leg_quoteVolume, short_leg_quoteVolume)
    quoteVolume_Weight = np.floor(quoteVolume_min/quoteVolume_min.sum()*100)/100
    
    if (quoteVolume_Weight.round(1) > 0).sum() == initial_number_of_pairs:
        print(f"Number of pairs decided: {number_of_pairs}")
        break
    number_of_pairs += 1
    if number_of_pairs > len(analyzer.opt_decision_sorted.columns):
        print("Error: Not enough pairs to invest")
        break

investing_pairs=analyzer.opt_decision_sorted.columns[np.where(quoteVolume_Weight>0)]
long_leg=pd.MultiIndex.from_tuples(investing_pairs.get_level_values(1))
short_leg=pd.MultiIndex.from_tuples(investing_pairs.get_level_values(2))
long_leg_quoteVolume=analyzer.FundingRate_Whole.loc[long_leg,'quoteVolume'].values
short_leg_quoteVolume=analyzer.FundingRate_Whole.loc[short_leg,'quoteVolume'].values
quoteVolume_min=np.minimum(long_leg_quoteVolume, short_leg_quoteVolume)
quoteVolume_Weight=np.floor(quoteVolume_min/quoteVolume_min.sum()*100)/100
# Step 2: 결정된 number_of_pairs로 나머지 계산 수행

order_quantity = pd.DataFrame(columns=investing_pairs)
orderType = investing_pairs.get_level_values(0)
long_leg_exchange = long_leg.get_level_values(2)
long_leg_quoteCoin = long_leg.get_level_values(1)
short_leg_exchange = short_leg.get_level_values(2)
short_leg_quoteCoin = short_leg.get_level_values(1)

try:
    order_available_cash_USDC = fetcher.magin_dashboards.loc['order_available_cash[USDC]'].drop(['BNB'],level=1)
    order_available_cash_USDC_sum = order_available_cash_USDC.sum()
except:
    order_available_cash_USDC = fetcher.magin_dashboards.loc['order_available_cash[USDC]']
    order_available_cash_USDC_sum = order_available_cash_USDC.sum()

order_quantity.loc['long_leg_account'] = list(zip(long_leg_exchange, long_leg_quoteCoin))
order_quantity.loc['long_leg_account_order_available_cash_current[USDC]'] = order_available_cash_USDC[order_quantity.loc['long_leg_account'].values].values
order_quantity.loc['short_leg_account'] = list(zip(short_leg_exchange, short_leg_quoteCoin))
order_quantity.loc['short_leg_account_order_available_cash_current[USDC]'] = order_available_cash_USDC[order_quantity.loc['short_leg_account'].values].values

order_quantity.loc['long_leg_quoteVolume'] = long_leg_quoteVolume
order_quantity.loc['short_leg_quoteVolume'] = short_leg_quoteVolume
order_quantity.loc['quoteVolume_min'] = quoteVolume_min
order_quantity.loc['quoteVolume_Weight'] = quoteVolume_Weight

order_quantity.loc['allocated_margin_for_Each_pair[USDC]'] = np.floor(order_quantity.loc['quoteVolume_Weight']*order_available_cash_USDC_sum/2)

margin_subset = order_quantity.loc[['allocated_margin_for_Each_pair[USDC]','long_leg_account_order_available_cash_current[USDC]','short_leg_account_order_available_cash_current[USDC]']].copy()
margin_subset
order_amount_list = []
# for th_pair in range(len(investing_pairs)):
th_pair=0
order_amount = margin_subset.iloc[:,th_pair].min()
order_amount_list.append(order_amount)
margin_subset_long_leg = margin_subset.loc['long_leg_account_order_available_cash_current[USDC]'].values.copy()
margin_subset_short_leg = margin_subset.loc['short_leg_account_order_available_cash_current[USDC]'].values.copy()

#deduction of margin from long_pair
deduction_of_margin_From_LongPair_For_LongPair_location = (order_quantity.loc['long_leg_account'].iloc[th_pair]==order_quantity.loc['long_leg_account']).values
deduction_of_margin_From_LongPair_For_ShortPair_location = (order_quantity.loc['long_leg_account'].iloc[th_pair]==order_quantity.loc['short_leg_account']).values
#deduction of margin from short_pair
deduction_of_margin_From_ShortPair_For_LongPair_location = (order_quantity.loc['short_leg_account'].iloc[th_pair]==order_quantity.loc['long_leg_account']).values
deduction_of_margin_From_ShortPair_For_ShortPair_location = (order_quantity.loc['short_leg_account'].iloc[th_pair]==order_quantity.loc['short_leg_account']).values

location_of_LongPair_to_change=(deduction_of_margin_From_LongPair_For_LongPair_location | deduction_of_margin_From_ShortPair_For_LongPair_location)
location_of_ShortPair_to_change=(deduction_of_margin_From_LongPair_For_ShortPair_location | deduction_of_margin_From_ShortPair_For_ShortPair_location)
location_of_LongPair_to_change[:th_pair+1] = False
location_of_ShortPair_to_change[:th_pair+1] = False
print(location_of_LongPair_to_change)
print(location_of_ShortPair_to_change)
# margin_subset_long_leg[location_of_LongPair_to_change] -= order_amount
# margin_subset_short_leg[location_of_ShortPair_to_change] -= order_amount
# margin_subset.loc['long_leg_account_order_available_cash_current[USDC]'] = margin_subset_long_leg
# margin_subset.loc['short_leg_account_order_available_cash_current[USDC]'] = margin_subset_short_leg

# margin_subset.loc['Min[allocatedMargin,availableCash][USDC]'] = order_amount_list
# margin_subset.rename(index={'long_leg_account_order_available_cash_current[USDC]':'long_leg_account_order_available_cash_BeforeOrder[USDC]'}, inplace=True)
# margin_subset.rename(index={'short_leg_account_order_available_cash_current[USDC]':'short_leg_account_order_available_cash_BeforeOrder[USDC]'}, inplace=True)


Number of pairs decided: 7
[False  True False False  True]
[False  True  True  True False]


In [133]:
margin_subset

LongMaker_ShortTaker  \
Long Leg                                           (POPCAT, USDC, hyperliquid)   
Short Leg                                               (POPCAT, USDT, bitget)   
allocated_margin_for_Each_pair[USDC]                                      6693   
long_leg_account_order_available_cash_current[U...                     5000.83   
short_leg_account_order_available_cash_current[...                19672.417045   

                                                          ShortMaker_LongTaker  \
Long Leg                                           (POPCAT, USDC, hyperliquid)   
Short Leg                                               (POPCAT, USDT, bitget)   
allocated_margin_for_Each_pair[USDC]                                      6693   
long_leg_account_order_available_cash_current[U...                     5000.83   
short_leg_account_order_available_cash_current[...                19672.417045   

                                                                             \
Long Leg                                               (MEW, USDT, binance)   
Short Leg                                          (MEW, USDC, hyperliquid)   
allocated_margin_for_Each_pair[USDC]                                   3201   
long_leg_account_order_available_cash_current[U...                      0.0   
short_leg_account_order_available_cash_current[...                  5000.83   

                                                       LongMaker_ShortTaker  \
Long Leg                                               (MEW, USDT, binance)   
Short Leg                                          (MEW, USDC, hyperliquid)   
allocated_margin_for_Each_pair[USDC]                                   3201   
long_leg_account_order_available_cash_current[U...                      0.0   
short_leg_account_order_available_cash_current[...                  5000.83   

                                                        ShortMaker_LongTaker  
Long Leg                                           (COMP, USDC, hyperliquid)  
Short Leg                                               (COMP, USDT, gateio)  
allocated_margin_for_Each_pair[USDC]                                    8730  
long_leg_account_order_available_cash_current[U...                   5000.83  
short_leg_account_order_available_cash_current[...               29153.81509

# Portfolio Tracker

In [168]:
# 단일 거래소의 포지션 데이터를 처리하는 함수
def process_exchange_positions(exchange_name: str, exchange_params: Dict) -> pd.DataFrame:
    # 거래소 객체 생성
    exchange = getattr(ccxt, exchange_name)(exchange_params)
    
    # 포지션 데이터 가져오기
    if exchange_name=='binance':
        positions_df = pd.DataFrame(exchange.fetchPositions())
        positions_df['leverage']=positions_df['notional']/positions_df['initialMargin']
        positions_df[['symbol', 'unrealizedPnl', 'contracts', 'contractSize', 'leverage', 'side', 'liquidationPrice']]
    
    else:
        positions_df = pd.DataFrame(exchange.fetchPositions())
    
    # notional_position_quantity 계산
    notional_position_quantity = (positions_df['contracts'].astype(float) * 
                                 positions_df['contractSize'].astype(float))
    notional_position_quantity = np.where(
        positions_df['side'] == 'short', 
        -notional_position_quantity, 
        notional_position_quantity
    ).astype('int64')
    
    # 인덱스 분리 및 ticker 처리
    index_split = positions_df['symbol'].str.replace(r'[:, /]', '|', regex=True).str.split('|', expand=True)
    index_split = index_split.iloc[:, :2]  # base_coin, quote_coin만 사용
    index_split.columns = ['base_coin', 'quote_coin']
    index_split['ticker'] = positions_df['symbol']
    
    # scaling_factor 계산
    scaling_factor = np.ones(len(index_split), dtype=int)
    base_coin = index_split['base_coin']
    scaling_factor[base_coin.str.startswith('k')] = 1000
    scaling_factor[base_coin.str.startswith('1M')] = 1000000
    numeric_match = base_coin.str.extract(r'^(?P<number>\d{2,})').dropna()
    if not numeric_match.empty:
        scaling_factor[numeric_match.index] = numeric_match['number'].astype(int).values
    
    # base_coin에서 접두어 제거
    index_k_or_number = base_coin.str.match(r'^(?:k|1M|\d{2,})')
    base_coin_cleaned = base_coin.str.replace(r'^(?:k|1M|\d{2,})', '', regex=True).where(
        index_k_or_number, base_coin
    )
    
    # 최종 데이터프레임 구성
    result_df = pd.DataFrame({
        'ticker': index_split['ticker'].values,
        'scaling_factor': scaling_factor,
        'notional_position_quantity': notional_position_quantity,
        'liquidationPrice': positions_df['liquidationPrice'].values,
        'leverage': positions_df['leverage'].values,
    }, index=pd.MultiIndex.from_arrays(
        [base_coin_cleaned, index_split['quote_coin'], [exchange_name] * len(index_split)],
        names=['base_coin', 'quote_coin', 'exchange']
    ))
    
    return result_df

# 멀티쓰레딩으로 모든 거래소 처리
def fetch_all_positions(exchange_configs: Dict) -> pd.DataFrame:
    position_handled_list = []
    
    with ThreadPoolExecutor(max_workers=min(len(exchange_configs), 8)) as executor:
        # 각 거래소 작업을 병렬로 실행
        future_to_exchange = {
            executor.submit(process_exchange_positions, name, params): name
            for name, params in exchange_configs.items()
        }
        # 결과 수집
        for future in future_to_exchange:
            try:
                position_handled_list.append(future.result())
            except Exception as e:
                print(f"Error processing {future_to_exchange[future]}: {e}")
    
    # 모든 결과를 하나로 결합
    return pd.concat(position_handled_list)

# exchange_configs는 기존 코드에서 제공된다고 가정
position_handled_df = fetch_all_positions(exchange_configs)
position_handled_df=position_handled_df[position_handled_df['notional_position_quantity']!=0]
position_handled_df['fundingRate']=FundingRate_Whole.loc[position_handled_df.index]['fundingRate']
position_handled_df['interval']=FundingRate_Whole.loc[position_handled_df.index]['interval']
position_handled_df['bid']=FundingRate_Whole.loc[position_handled_df.index]['bid_USDC_scaled']
position_handled_df['ask']=FundingRate_Whole.loc[position_handled_df.index]['ask_USDC_scaled']
position_handled_df['indexPrice']=FundingRate_Whole.loc[position_handled_df.index]['indexPrice']
position_handled_df['markPrice']=FundingRate_Whole.loc[position_handled_df.index]['markPrice']
position_handled_df['exchange']=position_handled_df.index.get_level_values(2)
position_direction=np.where(position_handled_df['notional_position_quantity']>0,'long','short')
position_handled_df=reorder_columns(position_handled_df,['ticker','scaling_factor','notional_position_quantity','leverage','fundingRate','interval','liquidationPrice','bid','ask'])

position_handled_df.index=[position_handled_df.index.get_level_values(0),position_direction]
########################################################################################################
# 보기 좋게 sorting
# 임시 열 추가
position_handled_df['abs_notional'] = position_handled_df['notional_position_quantity'].abs()  # 절댓값
position_handled_df['is_negative'] = position_handled_df['notional_position_quantity'] < 0     # 양수 여부


# 정렬: 절댓값 내림차순, 같은 절댓값 내에서 양수가 위로
position_handled_df = position_handled_df.sort_values(
    by=['abs_notional', 'is_negative'],
    ascending=[False, False]
)

# 임시 열 삭제
position_handled_df = position_handled_df.drop(columns=['abs_notional', 'is_negative'])

styled_df = position_handled_df.style.format({'bid': '{:.6f}', 'ask': '{:.6f}'})
display(styled_df)

Error processing bybit: 'contracts'
Error processing hyperliquid: 'contracts'


In [169]:
position_handled_df_unstack=position_handled_df.unstack()
fundingRate_LongShot_8H=position_handled_df_unstack['fundingRate'].fillna(0)*8/position_handled_df_unstack['interval'].fillna(8)
position_handled_df_unstack['funding rate spread 8H']=fundingRate_LongShot_8H['short']-fundingRate_LongShot_8H['long']

position_handled_df_unstack['position_Q_sum']=(position_handled_df_unstack['notional_position_quantity'].fillna(0)*position_handled_df_unstack['scaling_factor'].fillna(0)).sum(axis=1)

position_handled_df_unstack[('position_value','long')]=position_handled_df_unstack[('notional_position_quantity','long')]*position_handled_df_unstack[('markPrice','long')]
position_handled_df_unstack[('position_value','short')]=position_handled_df_unstack[('notional_position_quantity','short')]*position_handled_df_unstack[('markPrice','short')]
position_handled_df_unstack['long_bid_lower']=position_handled_df_unstack[('bid','long')]<position_handled_df_unstack[('bid','short')]
position_handled_df_unstack['long_ask_lower']=position_handled_df_unstack[('ask','long')]<position_handled_df_unstack[('ask','short')]
position_handled_df_unstack['long_markPrice_higher']=position_handled_df_unstack[('markPrice','long')]>position_handled_df_unstack[('markPrice','short')]
position_handled_df_unstack['funding rate spread 1Y']=position_handled_df_unstack['funding rate spread 8H']*3*365

position_handled_df_unstack[('Expected Income 1D','long')]=  -position_handled_df_unstack[('fundingRate','long')]*position_handled_df_unstack[('position_value','long')]*24/position_handled_df_unstack[('interval','long')].fillna(8)
position_handled_df_unstack[('Expected Income 1D','short')]= -position_handled_df_unstack[('fundingRate','short')]*position_handled_df_unstack[('position_value','short')]*24/position_handled_df_unstack[('interval','short')].fillna(8)
position_handled_df_unstack[('Expected Income 1D','whole')]= position_handled_df_unstack[('Expected Income 1D','long')].fillna(0)+position_handled_df_unstack[('Expected Income 1D','short')].fillna(0)



position_handled_df_unstack=reorder_columns(position_handled_df_unstack,['ticker','exchange','funding rate spread 8H','funding rate spread 1Y','Expected Income 1D','position_Q_sum','','long_bid_lower','long_ask_lower','long_markPrice_higher'],level=0)
position_handled_df_unstack


C:\Users\panda\AppData\Local\Temp/ipykernel_40520/3037817575.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fundingRate_LongShot_8H=position_handled_df_unstack['fundingRate'].fillna(0)*8/position_handled_df_unstack['interval'].fillna(8)
C:\Users\panda\AppData\Local\Temp/ipykernel_40520/3037817575.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  position_handled_df_unstack[('Expected Income 1D','whole')]= position_handled_df_unstack[('Expected Income 1D','long')].fillna(0)+position_handled_df_unstack[('Expected Income 1D','short')].fillna(0)


ticker                 exchange          \
                     long           short     long   short   
base_coin                                                    
ATH                   NaN   ATH/USDT:USDT      NaN  bitget   
DOG         DOG/USDT:USDT   DOG/USDT:USDT   bitget  gateio   
MOVE       MOVE/USDT:USDT  MOVE/USDT:USDT  binance  bitget   

          funding rate spread 8H funding rate spread 1Y Expected Income 1D  \
                                                                      long   
base_coin                                                                    
ATH                     0.000100               0.109500                NaN   
DOG                     0.000000               0.000000          -2.980359   
MOVE                    0.000396               0.433751          76.345263   

                                position_Q_sum  ...       bid            \
               short      whole                 ...      long     short   
base_coin                                       ...                       
ATH         0.876827   0.876827       -98310.0  ...       NaN  0.029726   
DOG         2.982162   0.001803            0.0  ...  0.001653  0.001652   
MOVE      -55.833624  20.511639            0.0  ...  0.410682  0.410282   

                ask           indexPrice           markPrice            \
               long     short       long     short      long     short   
base_coin                                                                
ATH             NaN  0.029746        NaN  0.029769       NaN   0.02973   
DOG        0.001654  0.001655   0.001653  0.001655  0.001653  0.001654   
MOVE       0.410782  0.410482   0.411083  0.410943  0.410478    0.4103   

          position_value             
                    long      short  
base_coin                            
ATH                  NaN -2922.7563  
DOG              9934.53   -9940.54  
MOVE           17240.076   -17232.6  

[3 rows x 35 columns]

In [170]:
allprint(position_handled_df_unstack)

ticker                 exchange          \
                     long           short     long   short   
base_coin                                                    
ATH                   NaN   ATH/USDT:USDT      NaN  bitget   
DOG         DOG/USDT:USDT   DOG/USDT:USDT   bitget  gateio   
MOVE       MOVE/USDT:USDT  MOVE/USDT:USDT  binance  bitget   

          funding rate spread 8H funding rate spread 1Y Expected Income 1D  \
                                                                      long   
base_coin                                                                    
ATH                     0.000100               0.109500                NaN   
DOG                     0.000000               0.000000          -2.980359   
MOVE                    0.000396               0.433751          76.345263   

                                position_Q_sum long_bid_lower long_ask_lower  \
               short      whole                                                
base_coin                                                                      
ATH         0.876827   0.876827       -98310.0          False          False   
DOG         2.982162   0.001803            0.0          False           True   
MOVE      -55.833624  20.511639            0.0          False          False   

          long_markPrice_higher scaling_factor        \
                                          long short   
base_coin                                              
ATH                       False            NaN   1.0   
DOG                       False            1.0   1.0   
MOVE                       True            1.0   1.0   

          notional_position_quantity            leverage       fundingRate  \
                                long      short     long short        long   
base_coin                                                                    
ATH                              NaN   -98310.0      NaN   3.0         NaN   
DOG                        6010000.0 -6010000.0      3.0   3.0     0.00005   
MOVE                         42000.0   -42000.0      3.0   3.0   -0.000738   

                   interval       liquidationPrice                 bid  \
             short     long short             long     short      long   
base_coin                                                                
ATH        0.00005      NaN   4.0              NaN  0.047136       NaN   
DOG        0.00005      4.0   4.0         0.001174  0.002686  0.001653   
MOVE      -0.00054      4.0   4.0         0.289879  0.681615  0.410682   

                          ask           indexPrice           markPrice  \
              short      long     short       long     short      long   
base_coin                                                                
ATH        0.029726       NaN  0.029746        NaN  0.029769       NaN   
DOG        0.001652  0.001654  0.001655   0.001653  0.001655  0.001653   
MOVE       0.410282  0.410782  0.410482   0.411083  0.410943  0.410478   

                    position_value             
              short           long      short  
base_coin                                      
ATH         0.02973            NaN -2922.7563  
DOG        0.001654        9934.53   -9940.54  
MOVE         0.4103      17240.076   -17232.6

# SL, TP Order 가져오기

In [ ]:
import requests
import time
import hmac
import hashlib
from urllib.parse import urlencode
import base64

exchange_name='bitget'
exchange_params=exchange_configs[exchange_name]
API_KEY=exchange_params['apiKey']
SECRET_KEY=exchange_params['secret']
PASSPHRASE=exchange_params['password']

In [ ]:
BASE_URL = "https://api.bitget.com"

def generate_signature(timestamp, method, endpoint, params):
    """Bitget API 서명 생성"""
    query_string = urlencode(sorted(params.items())) if params else ""
    message = f"{timestamp}{method.upper()}{endpoint}{'?' + query_string if query_string else ''}"
    signature = hmac.new(
        SECRET_KEY.encode('utf-8'),
        message.encode('utf-8'),
        hashlib.sha256
    ).digest()
    return base64.b64encode(signature).decode('utf-8')

# API 요청
endpoint = "/api/v2/mix/order/orders-plan-pending"
params = {
    "planType": "profit_loss",
    "productType": "usdt-futures"
}
timestamp = str(int(time.time() * 1000))  # 밀리초 단위 타임스탬프
method = "GET"

# 서명 생성
signature = generate_signature(timestamp, method, endpoint, params)

# 헤더 설정
headers = {
    "ACCESS-KEY": API_KEY,
    "ACCESS-SIGN": signature,
    "ACCESS-TIMESTAMP": timestamp,
    "ACCESS-PASSPHRASE": PASSPHRASE,
    "Content-Type": "application/json",
    "locale": "en-US"
}

# 요청 보내기
try:
    response = requests.get(
        f"{BASE_URL}{endpoint}",
        headers=headers,
        params=params
    )
    
    # 응답 처리
    if response.status_code == 200:
        print("Response:", response.json())
    else:
        print(f"Error: {response.status_code}, {response.text}")
except requests.exceptions.RequestException as e:
    print(f"Request error: {str(e)}")

Response: {'code': '00000', 'msg': 'success', 'requestTime': 1742310480036, 'data': {'entrustedList': [{'planType': 'loss_plan', 'symbol': 'BMTUSDT', 'size': '28589', 'orderId': '1285882361561055233', 'clientOid': '1285882361561055232', 'price': '0.194', 'executePrice': '0.194', 'callbackRatio': '', 'triggerPrice': '0.1966', 'triggerType': 'mark_price', 'planStatus': 'live', 'side': 'buy', 'posSide': 'long', 'marginCoin': 'USDT', 'marginMode': '', 'enterPointSource': 'WEB', 'tradeSide': 'close', 'posMode': 'hedge_mode', 'orderType': 'limit', 'stopSurplusTriggerPrice': None, 'stopSurplusExecutePrice': None, 'stopSurplusTriggerType': None, 'stopLossTriggerPrice': '0.1966', 'stopLossExecutePrice': None, 'stopLossTriggerType': 'mark_price', 'cTime': '1742304166212', 'uTime': '1742304166212'}, {'planType': 'profit_plan', 'symbol': 'BMTUSDT', 'size': '28589', 'orderId': '1285882361514917889', 'clientOid': '1285882361514917888', 'price': '0.3243', 'executePrice': '0.3243', 'callbackRatio': ''

# Index Price

In [ ]:
import ccxt
import time

# Binance 거래소 객체 생성
exchange = ccxt.binance({
    'enableRateLimit': True,  # API 요청 속도 제한 준수
})

# 영구 선물 시장 설정
exchange.options['defaultType'] = 'future'

# 1분 단위 OHLCV 데이터 가져오기
symbol = 'BTCUSDT'  # 예: BTC/USDT 영구 선물
timeframe = '1m'    # 1분 단위

In [ ]:
ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=1)
ohlcv

[[1742134140000, 82765.3, 82789.3, 82735.9, 82755.0, 77.695]]

In [ ]:
import ccxt
import time

# Binance 객체 생성
exchange = ccxt.binance({
    'enableRateLimit': True,
})

# 선물 시장 설정
exchange.options['defaultType'] = 'future'

# 심볼 설정
symbol = 'BTC/USDT:USDT'

In [ ]:
print(exchange.fetch_index_ohlcv(symbol, '1h', limit=1))
print(exchange.fetch_ohlcv(symbol, '1h', limit=1))

[[1742133600000, 82598.69357143, 82944.97547619, 82581.13690476, 82680.8952381, 0.0]]
[[1742133600000, 82561.0, 82916.2, 82537.3, 82609.1, 2031.992]]


In [ ]:


while True:
    try:
        # 마크 가격 및 인덱스 가격 정보 가져오기
        mark_price_data = exchange.fetch_mark_price(symbol)
        
        # 인덱스 가격은 거래소마다 키가 다를 수 있음 (Binance는 별도 호출 필요)
        ticker = exchange.fetch_funding_rate(symbol)  # 펀딩률 데이터에서 인덱스 가격 추출 가능
        index_price = exchange.fetch_index_price(symbol) if hasattr(exchange, 'fetch_index_price') else '별도 API 필요'
        
        # 결과 출력
        print(f"마크 가격: {mark_price_data['markPrice']}")
        print(f"인덱스 가격: {index_price}")
        
        # 1분 대기
        time.sleep(60)
    except AttributeError:
        print("이 거래소는 fetch_index_price를 지원하지 않습니다. 별도 API를 확인하세요.")
        break
    except Exception as e:
        print(f"에러 발생: {e}")
        time.sleep(5)

인덱스 가격: None


In [ ]:


while True:
    try:
        # OHLCV 데이터 요청
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=1)
        timestamp = ohlcv[0][0]  # 타임스탬프
        close_price = ohlcv[0][4]  # 종가 (인덱스 가격 대신 OHLCV 종가 사용)

        # 결과 출력
        print(f"시간: {exchange.iso8601(timestamp)}, 종가: {close_price}")

        # 1분 대기
        time.sleep(60)
    except Exception as e:
        print(f"에러 발생: {e}")
        time.sleep(5)  # 에러 시 잠시 대기 후 재시도

시간: 2025-03-16T14:09:00.000Z, 종가: 82780.5


In [ ]:
allprint(position_handled_df_unstack)

ticker                     exchange          \
                         long               short     long   short   
base_coin                                                            
ATH                       NaN       ATH/USDT:USDT      NaN  bitget   
AVL             AVL/USDT:USDT       AVL/USDT:USDT    bybit  bitget   
DOG             DOG/USDT:USDT       DOG/USDT:USDT   bitget  gateio   
MOVE           MOVE/USDT:USDT      MOVE/USDT:USDT  binance  bitget   
SATS       1000SATS/USDT:USDT  1000SATS/USDT:USDT  binance  bitget   

          funding rate spread 8H funding rate spread 1Y Expected Income 1D  \
                                                                      long   
base_coin                                                                    
ATH                     0.000100               0.109500                NaN   
AVL                     0.000000               0.000000          -2.401812   
DOG                     0.000000               0.000000          -3.604197   
MOVE                    0.000183               0.200889          20.605327   
SATS                    0.000000               0.000000          -0.110025   

                               position_Q_sum long_bid_lower long_ask_lower  \
              short      whole                                                
base_coin                                                                     
ATH        1.061453   1.061453       -98310.0          False          False   
AVL        2.402379   0.000567            0.0          False           True   
DOG        3.604197   0.000000            0.0          False          False   
MOVE      -9.838886  10.766440            0.0          False          False   
SATS        0.11025   0.000225            0.0           True           True   

          long_markPrice_higher scaling_factor          \
                                          long   short   
base_coin                                                
ATH                       False            NaN     1.0   
AVL                       False            1.0     1.0   
DOG                       False            1.0     1.0   
MOVE                       True            1.0     1.0   
SATS                      False         1000.0  1000.0   

          notional_position_quantity            leverage       fundingRate  \
                                long      short     long short        long   
base_coin                                                                    
ATH                              NaN   -98310.0      NaN   3.0         NaN   
AVL                          18900.0   -18900.0      3.0   3.0    0.000025   
DOG                        6010000.0 -6010000.0      3.0   3.0     0.00005   
MOVE                         42000.0   -42000.0      3.0   3.0   -0.000176   
SATS                       3000000.0 -3000000.0      3.0   3.0     0.00005   

                    interval       liquidationPrice                     bid  \
              short     long short             long     short          long   
base_coin                                                                     
ATH         0.00005      NaN   4.0              NaN  0.047035           NaN   
AVL        0.000025      2.0   2.0         0.380100  0.717808  4.237000e-01   
DOG         0.00005      4.0   4.0         0.001174  0.002678  1.998000e-03   
MOVE      -0.000084      4.0   4.0         0.333552  0.689520  4.652000e-01   
SATS        0.00005      4.0   4.0              NaN  0.000166  1.222000e-07   

                                  ask               indexPrice            \
                  short          long         short       long     short   
base_coin                                                                  
ATH        3.597000e-02           NaN  3.599000e-02        NaN  0.036019   
AVL        4.233000e-01  4.238000e-01  4.239000e-01     0.4238   0.42437   
DOG        1.997000e-03  1.999000e-03  1.999000e-03   0.002005  0.002001   
MOVE       4.648000e-01  

In [ ]:
position_handled_df_unstack[['position_value','fundingRate']]

position_value            fundingRate          
                    long      short        long     short
base_coin                                                
ATH                  NaN -3538.1769         NaN   0.00005
AVL              8006.04   -8007.93    0.000025  0.000025
DOG             12013.99  -12013.99     0.00005   0.00005
MOVE             19542.6   -19521.6   -0.000176 -0.000084
SATS              366.75     -367.5     0.00005   0.00005

# Fetch Funding Rate History

In [ ]:
def fetch_all_funding_history(exchange, symbol, since, limit=500):
    """
    'since' 시점부터 현재까지, fetchFundingRateHistory를
    여러 번 호출해 모든 펀딩 레이트 이력을 수집.
    """
    all_data = []
    next_since = since

    while True:
        # 3-1) 호출
        records = exchange.fetchFundingRateHistory(symbol, since=next_since, limit=limit)

        if not records:
            # 더 이상 신규 데이터 없으면 종료
            break

        all_data.extend(records)

        # 3-2) 마지막 타임스탬프 업데이트
        #     records가 시간순으로 정렬되어 있다고 가정
        last_ts = records[-1]['timestamp']
        
        # # 만약 마지막 루프이거나 더 이상 진행할 수 없으면 종료
        if last_ts is None or len(records) < limit:
            break

        # 다음 since: 마지막 레코드 + 1ms
        next_since = last_ts + 1
        
    return all_data

In [ ]:
exchange=exchange_HL
since = exchange_HL.parse8601('2024-01-01T00:00:00Z')  # 2024년 1월 1일 이후
symbol='BTC/USDC:USDC'
funding_list=fetch_all_funding_history(exchange, symbol, since, limit=500)
df = pd.DataFrame(funding_list)
df.index=pd.to_datetime(df['datetime']).dt.floor('h')
df

,info,symbol,fundingRate,timestamp,datetime
datetime,,,,,
2024-01-01 00:00:00+00:00,"{'coin': 'BTC', 'fundingRate': '0.0000617991',...",BTC/USDC:USDC,0.000062,1704067200151,2024-01-01T00:00:00.151Z
2024-01-01 01:00:00+00:00,"{'coin': 'BTC', 'fundingRate': '0.0000400063',...",BTC/USDC:USDC,0.000040,1704070800006,2024-01-01T01:00:00.006Z
2024-01-01 02:00:00+00:00,"{'coin': 'BTC', 'fundingRate': '0.0000433791',...",BTC/USDC:USDC,0.000043,1704074400173,2024-01-01T02:00:00.173Z
2024-01-01 03:00:00+00:00,"{'coin': 'BTC', 'fundingRate': '0.0000501325',...",BTC/USDC:USDC,0.000050,1704078000037,2024-01-01T03:00:00.037Z
2024-01-01 04:00:00+00:00,"{'coin': 'BTC', 'fundingRate': '0.0000502852',...",BTC/USDC:USDC,0.000050,1704081600362,2024-01-01T04:00:00.362Z
...,...,...,...,...,...
2025-03-01 10:00:00+00:00,"{'coin': 'BTC', 'fundingRate': '-0.000017581',...",BTC/USDC:USDC,-0.000018,1740823200068,2025-03-01T10:00:00.068Z
2025-03-01 11:00:00+00:00,"{'coin': 'BTC', 'fundingRate': '-0.0000441958'...",BTC/USDC:USDC,-0.000044,1740826800017,2025-03-01T11:00:00.017Z
2025-03-01 12:00:00+00:00,"{'coin': 'BTC', 'fundingRate': '-0.000043231',...",BTC/USDC:USDC,-0.000043,1740830400049,2025-03-01T12:00:00.049Z


In [ ]:

# 2024년 12월 31일의 타임스탬프 (밀리초 단위)
END_DATE = int(datetime(2024, 1, 1, 0, 0, 0).timestamp() * 1000)

def fetch_funding_rate_history(symbol="BTCUSDT_UMCBL", product_type="USDT-FUTURES", page_size=100):
    base_url = "https://api.bitget.com/api/mix/v1/market/history-fundRate"
    all_data = []
    page_no = 1
    
    while True:
        params = {
            "symbol": symbol,
            "productType": product_type,
            "pageSize": page_size,
            "pageNo": page_no
        }
        
        # API 요청
        response = requests.get(base_url, params=params)
        data = response.json().get("data", [])
        
        # 데이터가 없으면 중단
        if not data:
            # print(f"No more data at page {page_no}")
            break
        
        # 데이터 추가
        all_data.extend(data)
        # print(f"Page {page_no} fetched, {len(data)} records")
        
        # 가장 오래된 데이터의 settleTime 확인
        last_settle_time = pd.to_numeric(data[-1]["settleTime"])
        if last_settle_time < END_DATE:
            print(f"{symbol} Reached data before END_DATE at page {page_no}")
            break
        
        # 다음 페이지로 이동
        page_no += 1
        
        # API 제한을 피하기 위해 0.1초 대기
        #time.sleep(0.1)
    
    # DataFrame 생성
    df = pd.DataFrame(all_data)
    df['settleTime'] = pd.to_datetime(pd.to_numeric(df['settleTime']), unit='ms', utc=True)
    
    # 2024년까지만 필터링
    
    return df

# 데이터 가져오기
df = fetch_funding_rate_history(page_size=100)
print(f"Total records fetched: {len(df)}")

# 결과 출력
display(df)

BTCUSDT_UMCBL Reached data before END_DATE at page 13
Total records fetched: 1300


,symbol,fundingRate,settleTime
0,BTCUSDT,-0.000084,2025-03-01 08:00:00+00:00
1,BTCUSDT,0.000001,2025-03-01 00:00:00+00:00
2,BTCUSDT,-0.000004,2025-02-28 16:00:00+00:00
3,BTCUSDT,0.000122,2025-02-28 08:00:00+00:00
4,BTCUSDT,0.000122,2025-02-28 00:00:00+00:00
...,...,...,...
1295,BTCUSDT,0.000179,2023-12-25 16:00:00+00:00
1296,BTCUSDT,0.000108,2023-12-25 08:00:00+00:00
1297,BTCUSDT,0.000101,2023-12-25 00:00:00+00:00
1298,BTCUSDT,0.000102,2023-12-24 16:00:00+00:00
